### Testing

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import html5lib

In [2]:
!pip install html5lib

#### Link

In [4]:
#example url
url="https://www.kompas.com/tag/kurikulum-merdeka?type=all"
url

'https://www.kompas.com/tag/kurikulum-merdeka?type=all'

List keyword untuk di search (and periods)
- “pemilu” (20180417 - 20190417, 20231127 - 20241127) 
- “pilpres” (20180417 - 20190417, 20231127 - 20241127)
- “pemilihan umum” (20180417 - 20190417, 20231127 - 20241127)
- “pemilihan presiden” (20180417 - 20190417, 20231127 - 20241127)
- “Jokowi” (20180417 - 20190417, 20231127 - 20241127)
- ”Prabowo” (20180417 - 20190417, 20231127 - 20241127)
- “Sandi Uno” (20180417 - 20190417)
- “Ma’ruf Amin” (20180417 - 20190417)
- “Gibran” (20231127 - 20241127)
- “Anies Baswedan” (20231127 - 20241127)
- “Muhaimin Iskandar” (20231127 - 20241127)
- “Mahfud MD” (20231127 - 20241127)
- “Ganjar Pranowo” (20231127 - 20241127)

List Media:
- Kompas.com (bisa - otw)
- Detik.com
- iNews.id
- TVOneNews.com
- Metrotvnews.com
- Liputan6.com
- Beritasatu.com
- Republika.co.id
- Tempo.co
- Tirto.id

## Kompas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        # Add other headers as needed
    }
def ambil_detail_berita(url):
    try:
        res = requests.get(url, timeout=10, headers=headers)
        if res.status_code == 200:
            soup = BeautifulSoup(res.content, "html.parser")

        # Ambil judul
            judul_tag = soup.find("h1")
            judul = judul_tag.get_text(strip=True) if judul_tag else ""

        # Ambil isi artikel
            isi_paragraf = soup.find_all("p")
            isi_berita = "\n".join(p.get_text(strip=True) for p in isi_paragraf)

            return judul, isi_berita
        else:
            print("either 403 or 404")
            return "", ""
    except Exception as e:
        print(f" Gagal ambil {url}: {e}")
        return "", ""

def scrape_semua_halaman(max_page=395):
    base_url = "https://search.kompas.com/search?q=prabowo&site_id=all&start_date=2023-11-27&end_date=2024-11-27&page={}"
    data_berita = []
    
    for page in range(1, max_page + 1):
        print(f"\n🔎 Scraping halaman {page}...")
        res = requests.get(base_url.format(page),headers=headers)
        soup = BeautifulSoup(res.content, "html.parser")

        article_items = soup.find_all("div", class_="articleItem")

        for idx, item in enumerate(article_items, 1):
            a_tag = item.find("a", class_="article-link")
            tgl_tag = item.find("div", class_="articlePost-date")

            if a_tag:
                link = a_tag['href']
                tanggal = tgl_tag.get_text(strip=True) if tgl_tag else "Tidak ada tanggal"

                judul, isi = ambil_detail_berita(link)
                data_berita.append({
                    "URL": link,
                    "Tanggal": tanggal,
                    "Judul": judul,
                    "Isi Berita": isi
                })

                print(f" Artikel {link} di halaman {page} berhasil diambil.")
                time.sleep(1)  # Hindari beban server

    #print(data_berita[0])
    # Simpan ke Excel
    df = pd.DataFrame(data_berita)
    df.to_excel("kompas_prabowo2024.xlsx", index=False)
    print(f"\n Data berhasil disimpan ke dengan {len(df)} artikel.")

if __name__ == "__main__":
    scrape_semua_halaman()


🔎 Scraping halaman 1...
 Artikel https://nasional.kompas.com/read/2024/10/08/13144751/menimbang-kabinet-gemuk-prabowo di halaman 1 berhasil diambil.
 Artikel https://nasional.kompas.com/read/2024/01/10/06162811/dramaturgi-politik-prabowo-anies di halaman 1 berhasil diambil.
 Artikel https://www.kompas.com/stori/read/2024/02/28/185721179/pemberhentian-prabowo-dari-militer di halaman 1 berhasil diambil.
 Artikel https://regional.kompas.com/read/2024/01/09/170610378/prabowo-kita-tim-jokowi di halaman 1 berhasil diambil.
 Artikel https://nasional.kompas.com/read/2024/11/12/13402181/tantangan-kelembagaan-kabinet-prabowo di halaman 1 berhasil diambil.
 Artikel https://nasional.kompas.com/read/2024/04/12/06421661/prabowo-terus-upayakan-rekonsiliasi di halaman 1 berhasil diambil.
 Artikel https://www.kompas.com/cekfakta/read/2023/11/29/143223082/hoaks-menhan-prabowo-dinonaktifkan di halaman 1 berhasil diambil.
 Artikel https://money.kompas.com/read/2024/10/22/062226526/tantangan-ekonomi-pemer

##  Detik

In [20]:
import requests
from bs4 import BeautifulSoup

def ambil_isi_berita_detik(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        # Add other headers as needed
    }
        res = requests.get(url, timeout=10, headers=headers)
        soup = BeautifulSoup(res.content, "html.parser")

        # Coba semua kemungkinan lokasi isi artikel
        kontainer = soup.find("div", class_="detail__body-text itp_bodycontent")
        if not kontainer:
            kontainer = soup.find("div", class_="itp_bodycontent")
        if not kontainer:
            print("❌ Tidak ditemukan isi berita.")
            return ""

        paragraf_list = kontainer.find_all("p")
        isi_berita = "\n".join(p.get_text(strip=True) for p in paragraf_list)

        return isi_berita
    except Exception as e:
        print(f"❌ Gagal ambil isi {url}: {e}")
        return ""

# Contoh satu URL artikel Detik
url = "https://www.detik.com/edu/detikpedia/d-7616888/sejarah-kurikulum-di-indonesia-dari-kurikulum-1947-hingga-kurikulum-merdeka"
isi = ambil_isi_berita_detik(url)
print(isi[:1000])


Pernah dengar ungkapan, "ganti menteri, ganti kurikulum"? Kalimat ini kerap kali dilontarkan masyarakat ketika terjadi pergantian pemerintahan yang diikuti dengan perubahan kurikulum pendidikan. Seperti pada momen saat ini, Nadiem Makarim telah digantikan oleh tiga menteri baru yang menangani bidang pendidikan dan budaya.
Selama ini, pergantian pemerintahan memang berperan besar dalam menentukan arah pendidikan melalui pembuatan kurikulum baru. Namun, kecenderungan pemerintah untuk mengganti kurikulum setiap kali terjadi pergantian menteri pendidikan, kemudian melahirkan frasa "ganti menteri, ganti kurikulum".
Lantas, seberapa sering sebenarnya kurikulum diganti? Simak sejarah perkembangan kurikulum dari masa ke masa berikut ini.
SCROLL TO CONTINUE WITH CONTENT
Dalam studi berjudul "Sejarah Kurikulum di Indonesia" oleh Alhamuddin yang terbit pada tahun 2014, dikutip Selasa (29/10/2024), kurikulum di Indonesia telah ada sejak 1947. Berikut sejarah perkembangannya.
Kurikulum "rencana pel

In [16]:
import requests
from bs4 import BeautifulSoup
import time

def ambil_isi_berita(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.content, "html.parser")

        paragraf = soup.find_all("p")
        isi = "\n".join(p.get_text(strip=True) for p in paragraf)

        return isi
    except Exception as e:
        print(f"❌ Gagal ambil isi dari {url}: {e}")
        return ""

def scrape_detik(query="kurikulum+merdeka", max_page=25):
    for page in range(1, max_page + 1):
        url = f"https://www.detik.com/search/searchall?query={query}&siteid=2&source_kanal=true&sortby=time&page={page}"
        print(f"\n🔍 Halaman {page} — {url}")
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")

        artikel_list = soup.find_all("article", class_="list-content__item")

        if not artikel_list:
            print("🚫 Tidak ada artikel ditemukan. Selesai.")
            break

        for idx, artikel in enumerate(artikel_list, 1):
            judul_tag = artikel.find("h3", class_="media__title")
            tanggal_tag = artikel.find("div", class_="media__date")

            if judul_tag and judul_tag.a:
                judul = judul_tag.get_text(strip=True)
                link = judul_tag.a['href']
                tanggal = tanggal_tag.get_text(strip=True) if tanggal_tag else "Tidak ada tanggal"
                isi = ambil_isi_berita(link)

                print(f"\n📄 Artikel #{idx} di Halaman {page}")
                print(f"🗓️ Tanggal : {tanggal}")
                print(f"🔗 URL     : {link}")
                print(f"📌 Judul   : {judul}")
                print(f"📝 Isi     :\n{isi[:500]}...")  # batasi print isi 500 karakter

                time.sleep(1)  # supaya tidak terlalu cepat

if __name__ == "__main__":
    scrape_detik()


🔍 Halaman 1 — https://www.detik.com/search/searchall?query=kurikulum+merdeka&siteid=2&source_kanal=true&sortby=time&page=1

📄 Artikel #1 di Halaman 1
🗓️ Tanggal : Sabtu, 12 Apr 2025 17:00 WIB
🔗 URL     : https://www.detik.com/edu/sekolah/d-7865840/penjurusan-di-sma-belum-genap-setahun-dimatikan-menteri-muti-mau-hidupkan-lagi
📌 Judul   : Penjurusan di SMA Belum Genap Setahun 'Dimatikan', Menteri Mu'ti Mau Hidupkan Lagi
📝 Isi     :
ADVERTISEMENT
Rencana menghidupkan kembali sistem penjurusan di tingkat sekolah menengah atas (SMA) mengemuka. Padahal, kebijakan penghapusan sistem penjurusan IPA, IPS, dan Bahasa itu belum sampai setahun dijalankan secara resmi.
Niat untuk menjalankan kembali penjurusan dikemukakan Menteri Pendidikan Dasar dan Menengah Abdul Muti dalam Halalbihalal bersama Forum Wartawan Pendidikan di Perpustakaan Kemendikdasmen, Jl Jenderal Sudirman, Jakarta, Jumat (11/4/2025).
"Jurusan akan kita hidupkan lag...

📄 Artikel #2 di Halaman 1
🗓️ Tanggal : Jumat, 01 Nov 2024 11

KeyboardInterrupt: 

In [7]:
#mungkin ini
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def ambil_isi_berita(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.content, "html.parser")

        paragraf = soup.find_all("p")
        isi = "\n".join(p.get_text(strip=True) for p in paragraf)

        return isi
    except Exception as e:
        print(f"❌ Gagal ambil isi dari {url}: {e}")
        return ""

def scrape_detik_to_excel(query="kurikulum+merdeka", max_page=200, output_file="detik_articles.xlsx"):
    semua_data = []

    for page in range(1, max_page + 1):
        url = f"https://www.detik.com/search/searchall?query={query}&siteid=2&source_kanal=true&sortby=time&page={page}"
        print(f"\n🔍 Mengambil halaman {page}...")
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")

        artikel_list = soup.find_all("article", class_="list-content__item")

        if not artikel_list:
            print("🚫 Tidak ada artikel ditemukan di halaman ini.")
            break

        for idx, artikel in enumerate(artikel_list, 1):
            judul_tag = artikel.find("h3", class_="media__title")
            tanggal_tag = artikel.find("div", class_="media__date")

            if judul_tag and judul_tag.a:
                judul = judul_tag.get_text(strip=True)
                link = judul_tag.a['href']
                tanggal = tanggal_tag.get_text(strip=True) if tanggal_tag else "Tidak ada tanggal"
                isi = ambil_isi_berita(link)

                semua_data.append({
                    "Tanggal": tanggal,
                    "Judul": judul,
                    "URL": link,
                    "Isi Berita": isi
                })

                print(f"✅ Artikel #{idx} ditambahkan dari halaman {page}")
                time.sleep(1)

    # Simpan ke Excel
    df = pd.DataFrame(semua_data)
    df.to_excel(output_file, index=False)
    print(f"\n📁 Selesai! {len(df)} artikel disimpan ke '{output_file}'")

if __name__ == "__main__":
    scrape_detik_to_excel()



🔍 Mengambil halaman 1...
✅ Artikel #1 ditambahkan dari halaman 1
✅ Artikel #2 ditambahkan dari halaman 1
✅ Artikel #3 ditambahkan dari halaman 1
✅ Artikel #4 ditambahkan dari halaman 1
✅ Artikel #5 ditambahkan dari halaman 1
✅ Artikel #6 ditambahkan dari halaman 1
✅ Artikel #7 ditambahkan dari halaman 1
✅ Artikel #8 ditambahkan dari halaman 1
✅ Artikel #9 ditambahkan dari halaman 1
✅ Artikel #10 ditambahkan dari halaman 1
✅ Artikel #11 ditambahkan dari halaman 1
✅ Artikel #12 ditambahkan dari halaman 1

🔍 Mengambil halaman 2...
✅ Artikel #1 ditambahkan dari halaman 2
✅ Artikel #2 ditambahkan dari halaman 2
✅ Artikel #3 ditambahkan dari halaman 2
✅ Artikel #4 ditambahkan dari halaman 2
✅ Artikel #5 ditambahkan dari halaman 2
✅ Artikel #6 ditambahkan dari halaman 2
✅ Artikel #7 ditambahkan dari halaman 2
✅ Artikel #8 ditambahkan dari halaman 2
✅ Artikel #9 ditambahkan dari halaman 2
✅ Artikel #10 ditambahkan dari halaman 2
✅ Artikel #11 ditambahkan dari halaman 2
✅ Artikel #12 ditambahk

In [2]:
import requests
from bs4 import BeautifulSoup

def ambil_isi_berita(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.content, "html.parser")
        paragraf = soup.find_all("p")
        isi = "\n".join(p.get_text(strip=True) for p in paragraf)
        return isi
    except Exception as e:
        return f"❌ Gagal ambil isi dari {url}: {e}"

def scrape_page_1():
    url = "https://www.detik.com/tag/kurikulum-merdeka/?sortby=time&page=1"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")

    artikel_list = soup.find_all("article")
    hasil = []

    for artikel in artikel_list:
        try:
            a_tag = artikel.find("a", href=True)
            if not a_tag:
                continue
            
            link = a_tag["href"]
            judul_tag = artikel.find("h2", class_="title")
            judul = judul_tag.get_text(strip=True) if judul_tag else "Tidak ada judul"
            tanggal_tag = artikel.find("span", class_="date")
            tanggal = tanggal_tag.get_text(strip=True) if tanggal_tag else "Tidak ada tanggal"
            isi = ambil_isi_berita(link)

            hasil.append({
                "Tanggal": tanggal,
                "Judul": judul,
                "URL": link,
                "Isi Berita": isi[:300] + "..."  # biar ga kepanjangan
            })
        except Exception as e:
            print(f"❌ Gagal memproses artikel: {e}")

    for item in hasil:
        print(f"Tanggal : {item['Tanggal']}")
        print(f"Judul   : {item['Judul']}")
        print(f"URL     : {item['URL']}")
        print(f"Isi     : {item['Isi Berita']}")
        print("="*80)

scrape_page_1()

Tanggal : detikJatimSelasa, 22 Apr 2025 14:51 WIB
Judul   : Pakar Pendidikan Jatim Dukung Rencana Jurusan SMA Dikembalikan
URL     : https://www.detik.com/jatim/berita/d-7880388/pakar-pendidikan-jatim-dukung-rencana-jurusan-sma-dikembalikan
Isi     : Wacana pengembalian jurusan IPA/IPS/Bahasa pada tingkat Sekolah Menengah Atas (SMA) mulai mencuat. Pakar pendidikan Jatim rupanya mendukung hal tersebut sebagai upaya untuk menggali potensi siswa.
"Potensi itu kan tentu satu sama lain berbeda. Untuk mengetahui potensi itu sebetulnya anak-anak dapat ...
Tanggal : detikEduSenin, 21 Apr 2025 09:00 WIB
Judul   : Suara Siswa, Pengamat, hingga Guru Jika Penjurusan IPA/IPS/Bahasa Berlaku Lagi
URL     : https://www.detik.com/edu/sekolah/d-7877600/suara-siswa-pengamat-hingga-guru-jika-penjurusan-ipa-ips-bahasa-berlaku-lagi
Isi     : ADVERTISEMENT
Ajeng, siswa kelas 11 di Depok sepakat jika diadakan kembali penjurusan di tingkat SMA.
"Jujur dari pengalaman di sekolah saya sendiri, guru belum terlalu

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def ambil_isi_berita(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.content, "html.parser")
        paragraf = soup.find_all("p")
        isi = "\n".join(p.get_text(strip=True) for p in paragraf)
        return isi
    except Exception as e:
        return f"❌ Gagal ambil isi dari {url}: {e}"

def scrape_detik_tag(query_tag="kurikulum-merdeka", max_page=31, output_file="detik_tag_kurikulum_merdeka.xlsx"):
    semua_data = []

    for page in range(1, max_page + 1):
        print(f"\n🔍 Mengambil halaman {page}...")
        url = f"https://www.detik.com/tag/{query_tag}/?sortby=time&page={page}"
        res = requests.get(url)
        soup = BeautifulSoup(res.content, "html.parser")

        artikel_list = soup.find_all("article")

        if not artikel_list:
            print("🚫 Tidak ada artikel ditemukan di halaman ini.")
            break

        for idx, artikel in enumerate(artikel_list, 1):
            try:
                a_tag = artikel.find("a", href=True)
                if not a_tag:
                    continue

                link = a_tag["href"]
                judul_tag = artikel.find("h2", class_="title")
                judul = judul_tag.get_text(strip=True) if judul_tag else "Tidak ada judul"
                tanggal_tag = artikel.find("span", class_="date")
                tanggal = tanggal_tag.get_text(strip=True) if tanggal_tag else "Tidak ada tanggal"
                isi = ambil_isi_berita(link)

                semua_data.append({
                    "Tanggal": tanggal,
                    "Judul": judul,
                    "URL": link,
                    "Isi Berita": isi
                })

                print(f"✅ Artikel #{idx} dari halaman {page} ditambahkan")
                time.sleep(1)

            except Exception as e:
                print(f"❌ Gagal memproses artikel di halaman {page}: {e}")

    # Simpan ke Excel
    df = pd.DataFrame(semua_data)
    df.to_excel(output_file, index=False)
    print(f"\n📁 Selesai! {len(df)} artikel disimpan ke '{output_file}'")

if __name__ == "__main__":
    scrape_detik_tag()



🔍 Mengambil halaman 1...
✅ Artikel #1 dari halaman 1 ditambahkan
✅ Artikel #2 dari halaman 1 ditambahkan
✅ Artikel #3 dari halaman 1 ditambahkan
✅ Artikel #4 dari halaman 1 ditambahkan
✅ Artikel #5 dari halaman 1 ditambahkan
✅ Artikel #6 dari halaman 1 ditambahkan
✅ Artikel #7 dari halaman 1 ditambahkan
✅ Artikel #8 dari halaman 1 ditambahkan
✅ Artikel #9 dari halaman 1 ditambahkan
✅ Artikel #10 dari halaman 1 ditambahkan
✅ Artikel #11 dari halaman 1 ditambahkan
✅ Artikel #12 dari halaman 1 ditambahkan
✅ Artikel #13 dari halaman 1 ditambahkan
✅ Artikel #14 dari halaman 1 ditambahkan
✅ Artikel #15 dari halaman 1 ditambahkan
✅ Artikel #16 dari halaman 1 ditambahkan

🔍 Mengambil halaman 2...
✅ Artikel #1 dari halaman 2 ditambahkan
✅ Artikel #2 dari halaman 2 ditambahkan
✅ Artikel #3 dari halaman 2 ditambahkan
✅ Artikel #4 dari halaman 2 ditambahkan
✅ Artikel #5 dari halaman 2 ditambahkan
✅ Artikel #6 dari halaman 2 ditambahkan
✅ Artikel #7 dari halaman 2 ditambahkan
✅ Artikel #8 dari hal

## Tribun

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Setup Chrome
options = Options()
options.add_argument("--headless")  # Jalan di background
driver = webdriver.Chrome(options=options)

# Buka halaman search
url = 'https://www.tribunnews.com/search?q=kurikulum+merdeka&cx=partner-pub-7486139053367666%3A4965051114&cof=FORID%3A10&ie=UTF-8&siteurl=www.tribunnews.com#gsc.tab=0&gsc.q=kurikulum%20merdeka&gsc.page=1'
driver.get(url)

time.sleep(5)  # Tunggu hasil search termuat

# Ambil HTML setelah dimuat
soup = BeautifulSoup(driver.page_source, 'html.parser')

results = soup.select('div.gsc-webResult.gsc-result')
data = []

for result in results:
    a_tag = result.select_one('.gs-title a')
    if not a_tag:
        continue

    judul = a_tag.text.strip()
    link = a_tag['href']

    try:
        # Masuk ke halaman artikel
        driver.get(link)
        time.sleep(3)
        detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Ambil tanggal
        tanggal_el = detail_soup.select_one('div.article__date, time, .date, .tanya__date')
        tanggal = tanggal_el.text.strip() if tanggal_el else 'Tidak ada tanggal'

        # Ambil isi
        paragraphs = detail_soup.select('div#article p')
        isi = ' '.join(p.text.strip() for p in paragraphs if p.text.strip())

    except Exception as e:
        tanggal = 'Gagal ambil'
        isi = 'Gagal ambil isi'

    data.append({
        'Judul': judul,
        'URL': link,
        'Tanggal': tanggal,
        'Isi_Berita': isi
    })

# Tutup browser
driver.quit()

# Print hasil
for i, item in enumerate(data, 1):
    print(f"{i}. {item['Judul']}")
    print(f"   URL     : {item['URL']}")
    print(f"   Tanggal : {item['Tanggal']}")
    print(f"   Isi     : {item['Isi_Berita'][:200]}...\n")


1. 50 Soal Ujian Sekolah IPS Kelas 9 2025 Kurikulum Merdeka dan ...
   URL     : https://www.tribunnews.com/pendidikan/2025/04/24/50-soal-ujian-sekolah-ips-kelas-9-2025-kurikulum-merdeka-dan-kunci-jawaban-us
   Tanggal : Tayang: Kamis, 24 April 2025 19:54 WIB
   Isi     : TRIBUNNEWS.COM - Kumpulan contoh soal Ujian Sekolah IPS Kelas 9 SMP/MTs 2025 Kurikulum Merdeka, lengkap dengan kunci jawabannya. Soal Ujian Sekolah IPS Kelas 9 2025 Kurikulum Merdeka ini terdiri dari ...

2. Kunci Jawaban PAI Kelas 11 Halaman 288 289 290 291 Kurikulum ...
   URL     : https://www.tribunnews.com/pendidikan/2025/04/18/kunci-jawaban-pai-kelas-11-halaman-288-289-290-291-kurikulum-merdeka-penilaian-pengetahuan-bab-9
   Tanggal : Tayang: Jumat, 18 April 2025 14:59 WIB
   Isi     : TRIBUNNEWS.COM - Inilah kunci jawaban Pendidikan Agama Islam atau PAI kelas 11 SMA/SMK halaman 288, 289, 290, 291 buku Kurikulum Merdeka. Kunci jawaban PAI Kelas XI halaman 288, 289, 290, 291 buku Kur...

3. Kunci Jawaban PAI Kela

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup headless Chrome
options = Options()
driver = webdriver.Chrome(options=options)

data = []

# Loop halaman 1 sampai 10
for page in range(1, 11):
    print(f"[INFO] Mengakses halaman {page}...")
    search_url = f"https://www.tribunnews.com/search?q=kurikulum+merdeka&cx=partner-pub-7486139053367666%3A4965051114&cof=FORID%3A10&ie=UTF-8&siteurl=www.tribunnews.com#gsc.tab=0&gsc.q=kurikulum%20merdeka&gsc.page={page}"
    driver.get(search_url)
    time.sleep(5)  # tunggu hasil search termuat

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    results = soup.select('div.gsc-webResult.gsc-result')

    for result in results:
        a_tag = result.select_one('.gs-title a')
        if not a_tag:
            continue

        judul = a_tag.text.strip()
        link = a_tag['href']

        try:
            driver.get(link)
            time.sleep(3)
            detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

            # Ambil tanggal
            tanggal_el = detail_soup.select_one('div.article__date, time, .date, .tanya__date')
            tanggal = tanggal_el.text.strip() if tanggal_el else 'Tidak ada tanggal'

            # Ambil isi
            paragraphs = detail_soup.select('div#article p')
            isi = ' '.join(p.text.strip() for p in paragraphs if p.text.strip())
            isi = isi if isi else 'Tidak ada isi'

        except Exception as e:
            tanggal = 'Gagal ambil'
            isi = 'Gagal ambil isi'

        data.append({
            'Judul': judul,
            'URL': link,
            'Tanggal': tanggal,
            'Isi_Berita': isi
        })

# Tutup browser
driver.quit()

# Simpan ke Excel
df = pd.DataFrame(data)
df.to_excel('tribun_kurikulum_merdeka.xlsx', index=False)
print("[SELESAI] Data berhasil disimpan ke 'tribun_kurikulum_merdeka.xlsx'")


[INFO] Mengakses halaman 1...
[INFO] Mengakses halaman 2...
[INFO] Mengakses halaman 3...
[INFO] Mengakses halaman 4...
[INFO] Mengakses halaman 5...
[INFO] Mengakses halaman 6...
[INFO] Mengakses halaman 7...
[INFO] Mengakses halaman 8...
[INFO] Mengakses halaman 9...
[INFO] Mengakses halaman 10...
[SELESAI] Data berhasil disimpan ke 'tribun_kurikulum_merdeka.xlsx'


## TVONEWNEWS

In [5]:
import requests
from bs4 import BeautifulSoup

# URL pencarian awal
search_url = 'https://www.tvonenews.com/cari?q=kurikulum%20merdeka'
headers = {'User-Agent': 'Mozilla/5.0'}

# Step 1: Ambil daftar artikel dari halaman pencarian
res = requests.get(search_url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
articles = soup.find_all('div', class_='article-list-row')

# Step 2: Loop setiap artikel dan ambil info + buka halaman detail
for article in articles:
    try:
        # Judul dan link
        judul_tag = article.find('a', class_='ali-title')
        judul = judul_tag.get_text(strip=True)
        link = judul_tag['href']

        # Tanggal
        tanggal_tag = article.find('li', class_='ali-date')
        tanggal = tanggal_tag.get_text(strip=True)

        # Deskripsi singkat
        deskripsi_tag = article.find('div', class_='ali-desc')
        deskripsi = deskripsi_tag.get_text(strip=True)

        # Step 3: Masuk ke halaman detail
        detail_res = requests.get(link, headers=headers)
        detail_soup = BeautifulSoup(detail_res.text, 'html.parser')

        # Ambil semua <p> dan gabungkan jadi isi lengkap
        isi_paragraf = detail_soup.find_all('p')
        isi_berita = ' '.join(p.get_text(strip=True) for p in isi_paragraf if p.get_text(strip=True))

        print("Judul    :", judul)
        print("URL      :", link)
        print("Tanggal  :", tanggal)
        print("Deskripsi:", deskripsi)
        print("Isi Berita:")
        print(isi_berita)
        print('-' * 100)

    except Exception as e:
        print("Gagal parsing artikel:", e)


Judul    : Mendikdasmen Abdul Mu'ti Akhirnya Buka Suara soal Nasib Kurikulum Merdeka, Termasuk Makna "Deep learning" yang Ramai di Media Sosial
URL      : https://www.tvonenews.com/religi/265323-mendikdasmen-abdul-muti-akhirnya-buka-suara-soal-nasib-kurikulum-merdeka-termasuk-makna-deep-learning-yang-ramai-di-media-sosial
Tanggal  : 9/11/2024 - 09:41
Deskripsi: Menteri Pendidikan Dasar dan Menengah (Mendikdasmen) Abdul Mu'ti akhirnya menjelaskan arti "deep learning" atau pembelajaran mendalam yang sebelumnya ramai di media sosial.
Isi Berita:
ADVERTISEMENT Jakarta, tvOnenews.com-Menteri Pendidikan Dasar dan Menengah(Mendikdasmen) Abdul Mu'ti akhirnya menjelaskan arti "deep learning"atau pembelajaran mendalam yang sebelumnya ramai di media sosial. Menurut Sekretaris Jenderal PP Muhammadiyah itu, "Deep Learning" adalah  pendekatan belajar untuk meningkatkan kapasitas siswa, bukankurikulum pendidikan."'Deep learning'itu bukan kurikulum. Itu pendekatan belajar," ujarnya saat ditemui usai a

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL pencarian
search_url = 'https://www.tvonenews.com/cari?q=kurikulum%20merdeka'
headers = {'User-Agent': 'Mozilla/5.0'}

# Ambil daftar artikel dari halaman pencarian
res = requests.get(search_url, headers=headers)
soup = BeautifulSoup(res.text, 'html.parser')
articles = soup.find_all('div', class_='article-list-row')

# List penampung hasil
data = []

for article in articles:
    try:
        # Judul dan link
        judul_tag = article.find('a', class_='ali-title')
        judul = judul_tag.get_text(strip=True)
        link = judul_tag['href']

        # Tanggal
        tanggal_tag = article.find('li', class_='ali-date')
        tanggal = tanggal_tag.get_text(strip=True)

        # Deskripsi singkat
        deskripsi_tag = article.find('div', class_='ali-desc')
        deskripsi = deskripsi_tag.get_text(strip=True)

        # Masuk ke halaman detail
        detail_res = requests.get(link, headers=headers)
        detail_soup = BeautifulSoup(detail_res.text, 'html.parser')

        # Ambil isi lengkap dari semua <p>
        isi_paragraf = detail_soup.find_all('p')
        isi_berita = ' '.join(p.get_text(strip=True) for p in isi_paragraf if p.get_text(strip=True))

        # Simpan ke list
        data.append({
            'Judul': judul,
            'URL': link,
            'Tanggal': tanggal,
            'Deskripsi': deskripsi,
            'Isi Berita': isi_berita
        })

    except Exception as e:
        print(f"Gagal parsing artikel: {e}")

# Simpan ke Excel
df = pd.DataFrame(data)
df.to_excel('tvone_kurikulum.xlsx', index=False)
print("Berhasil disimpan ke 'tvone_kurikulum.xlsx'")


Gagal parsing artikel: 'NoneType' object has no attribute 'get_text'
Gagal parsing artikel: 'NoneType' object has no attribute 'get_text'
Berhasil disimpan ke 'tvone_kurikulum.xlsx'


## CNN INDONESIA

In [7]:
import requests
from bs4 import BeautifulSoup

def ambil_isi_berita(url):
    try:
        res = requests.get(url, timeout=10)
        soup = BeautifulSoup(res.content, "html.parser")
        paragraf = soup.find_all("p")
        isi = "\n".join(p.get_text(strip=True) for p in paragraf)
        return isi
    except Exception as e:
        return f"Gagal ambil isi: {e}"

def scrape_cnn_page(page=1):
    url = f"https://www.cnnindonesia.com/search?query=kurikulum%20merdeka&page={page}"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    artikel_list = soup.find_all("article", class_="flex-grow")

    hasil = []
    for artikel in artikel_list:
        a_tag = artikel.find("a", href=True)
        if not a_tag:
            continue
        
        link = a_tag["href"]

        # Cek kalau ada h2
        h2_tag = a_tag.find("h2")
        if not h2_tag:
            continue
        judul = h2_tag.get_text(strip=True)

        # Cek tanggal
        tanggal_tag = a_tag.find("span", class_="text-cnn_black_light3")
        tanggal = tanggal_tag.get_text(strip=True) if tanggal_tag else "Tidak ada tanggal"
        
        isi = ambil_isi_berita(link)

        hasil.append({
            "Tanggal": tanggal,
            "Judul": judul,
            "URL": link,
            "Isi": isi[:300] + "..."  # biar isi ga kepanjangan
        })
    return hasil

# Coba scrape halaman 1
hasil = scrape_cnn_page(1)
for item in hasil:
    print(f"Tanggal : {item['Tanggal']}")
    print(f"Judul   : {item['Judul']}")
    print(f"URL     : {item['URL']}")
    print(f"Isi     : {item['Isi']}")
    print("=" * 80)


In [15]:
import requests
from bs4 import BeautifulSoup

url = "https://www.cnnindonesia.com/search?query=kurikulum%20merdeka&page=2"
headers = {
    "User-Agent": "Mozilla/5.0"
}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text, "html.parser")

articles = soup.find_all("article", class_="flex-grow")

for article in articles:
    a_tag = article.find("a")
    if a_tag:
        link = a_tag.get("href", "")
        h2 = a_tag.find("h2")
        tanggal_span = a_tag.find("span", class_="text-cnn_black_light3")

        if h2 and tanggal_span:
            judul = h2.get_text(strip=True)
            tanggal = tanggal_span.get_text(strip=True)

            # Ambil isi berita
            artikel_res = requests.get(link, headers=headers)
            artikel_soup = BeautifulSoup(artikel_res.text, "html.parser")
            isi_berita = ""

            konten = artikel_soup.find("div", class_="detail-text")
            if konten:
                paragraphs = konten.find_all("p")
                isi_berita = "\n".join([p.get_text(strip=True) for p in paragraphs])

            # Cetak hasil
            print("Judul:", judul)
            print("Tanggal:", tanggal)
            print("Link:", link)
            print("Isi Berita:\n", isi_berita[:500], "...")
            print("="*80)

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import requests

# Setup Selenium (headless mode bisa diaktifkan kalau mau)
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")  # opsional, biar gak muncul jendela browser

driver = webdriver.Chrome(options=chrome_options)

# URL target
url = "https://www.cnnindonesia.com/search?query=kurikulum%20merdeka&page=2"
driver.get(url)

# Tunggu sampai konten dimuat (manual wait untuk amannya)
time.sleep(5)  # atau pakai WebDriverWait untuk lebih pintar

# Ambil HTML yang sudah dimuat
html = driver.page_source
driver.quit()

# Parse dengan BeautifulSoup
soup = BeautifulSoup(html, "html.parser")
articles = soup.find_all("article", class_="flex-grow")

headers = {"User-Agent": "Mozilla/5.0"}

for article in articles:
    a_tag = article.find("a")
    if not a_tag:
        continue

    h2 = a_tag.find("h2")
    tanggal_span = a_tag.find("span", class_="text-cnn_black_light3")

    if not h2 or not tanggal_span:
        continue

    judul = h2.get_text(strip=True)
    tanggal = tanggal_span.get_text(strip=True)
    link = a_tag["href"]

    # Ambil isi berita
    try:
        res = requests.get(link, headers=headers)
        artikel_soup = BeautifulSoup(res.text, "html.parser")
        konten = artikel_soup.find("div", class_="detail-text")
        isi_berita = ""
        if konten:
            paragraphs = konten.find_all("p")
            isi_berita = "\n".join(p.get_text(strip=True) for p in paragraphs)

        # Cetak hasil
        print("Judul:", judul)
        print("Tanggal:", tanggal)
        print("Link:", link)
        print("Isi Berita:\n", isi_berita[:500], "...\n")
        print("="*80)

    except Exception as e:
        print(f"⚠️ Gagal ambil isi dari {link} karena {e}")


Judul: Disdik DKI Sebut Semua SMA di Jakarta Sudah Pakai Kurikulum Merdeka
Tanggal: • 9 bulan yang lalu
Link: https://www.cnnindonesia.com/nasional/20240719133339-20-1123242/disdik-dki-sebut-semua-sma-di-jakarta-sudah-pakai-kurikulum-merdeka
Isi Berita:
 Dinas Pendidikan (Disdik)DKI Jakartamenyebut seluruh Sekolah Menengah Atas (SMA) di Jakarta telah mengimplementasikan Kurikulum Merdeka pada tahun ajaran 2024/2025.
"Pada tahun pelajaran 2024/2025 seluruh SMA telah mengimplikasikan kurikulum merdeka," kata Pelaksana Tugas (Plt) Kepala Dinas Pendidikan DKI Budi Awaluddin, Jumat (19/7).

ADVERTISEMENT
SCROLL TO CONTINUE WITH CONTENT
Budi menjelaskan struktur kurikulum pada kelas XI terdiri dari mata pelajaran umum dan mata pelajaran pilihan berdas ...

Judul: Cerita Gibran Bersurat ke Nadiem soal Merdeka Belajar tapi Tak Dibalas
Tanggal: • 5 bulan yang lalu
Link: https://www.cnnindonesia.com/nasional/20241111111343-32-1165188/cerita-gibran-bersurat-ke-nadiem-soal-merdeka-belajar-tapi-tak

In [3]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Inisialisasi driver
chrome_options = Options()
chrome_options.add_argument("--start-maximized")
# chrome_options.add_argument("--headless")  # uncomment kalau mau headless

driver = webdriver.Chrome(options=chrome_options)

headers = {"User-Agent": "Mozilla/5.0"}

# Simpan data di list
data_berita = []

# Loop halaman 1 - 100
for page in range(1, 11):
    print(f"🔎 Scraping halaman {page}...")
    url = f"https://www.cnnindonesia.com/search?query=kurikulum%20merdeka&result_type=latest&fromdate=01/02/2022&todate=08/05/2025&page={page}"
    driver.get(url)
    time.sleep(5)  # tunggu konten muncul

    soup = BeautifulSoup(driver.page_source, "html.parser")
    articles = soup.find_all("article", class_="flex-grow")

    for article in articles:
        a_tag = article.find("a")
        if not a_tag:
            continue

        h2 = a_tag.find("h2")
        tanggal_span = a_tag.find("span", class_="text-cnn_black_light3")

        if not h2 or not tanggal_span:
            continue

        judul = h2.get_text(strip=True)
        tanggal = tanggal_span.get_text(strip=True)
        link = a_tag["href"]

        # Ambil isi artikel
        try:
            res = requests.get(link, headers=headers)
            artikel_soup = BeautifulSoup(res.text, "html.parser")

            # Ambil tanggal dari dalam artikel
            tanggal_div = artikel_soup.find("div", class_="text-cnn_grey text-sm mb-4")
            tanggal = tanggal_div.get_text(strip=True) if tanggal_div else "Tanggal tidak ditemukan"

            # Ambil isi berita
            konten = artikel_soup.find("div", class_="detail-text")
            isi_berita = ""
            if konten:
                paragraphs = konten.find_all("p")
                isi_berita = "\n".join(p.get_text(strip=True) for p in paragraphs)

            # Tambahkan ke list
            data_berita.append({
                "Judul": judul,
                "Tanggal": tanggal,
                "Link": link,
                "Isi Berita": isi_berita
            })

        except Exception as e:
            print(f"⚠️ Gagal ambil isi dari {link} karena {e}")


driver.quit()

# Simpan ke Excel
df = pd.DataFrame(data_berita)
df.to_excel("CNN_Kurikulum Merdeka.xlsx", index=False)
print("✅ Selesai! Data disimpan ke 'CNN_Kurikulum Merdeka.xlsx'")

🔎 Scraping halaman 1...
🔎 Scraping halaman 2...
🔎 Scraping halaman 3...
🔎 Scraping halaman 4...
🔎 Scraping halaman 5...
🔎 Scraping halaman 6...
🔎 Scraping halaman 7...
🔎 Scraping halaman 8...
🔎 Scraping halaman 9...
🔎 Scraping halaman 10...
✅ Selesai! Data disimpan ke 'CNN_Kurikulum Merdeka.xlsx'


## Liputan 6

In [1]:
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests
from tqdm import tqdm

# Ambil isi teks dari link artikel
def ambil_isi_liputan6(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        konten = soup.find("div", class_="article-content-body__item-content")
        if konten:
            return konten.get_text(separator="\n", strip=True)
        else:
            return "Isi tidak ditemukan"
    except Exception as e:
        return f"Gagal mengambil isi: {e}"

# Scraper untuk satu halaman Liputan6
def scrape_liputan6_page(page=1):
    url = f"https://www.liputan6.com/tag/kurikulum-merdeka?page={page}"

    # Setup Selenium headless browser
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(3)  # tunggu halaman termuat

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    artikel_list = soup.find_all("article", class_="articles--iridescent-list--text-item")
    print(f"🔍 Ditemukan {len(artikel_list)} artikel di halaman {page}")

    hasil = []
    for artikel in tqdm(artikel_list, desc="📄 Memproses artikel"):
        try:
            a_tag = artikel.find("a", class_="articles--iridescent-list--text-item__title-link")
            if not a_tag:
                continue
            link = a_tag["href"]
            judul = a_tag.get_text(strip=True)

            time_tag = artikel.find("time")
            tanggal = time_tag["title"] if time_tag and "title" in time_tag.attrs else "Tanggal tidak ditemukan"

            isi = ambil_isi_liputan6(link)

            hasil.append({
                "Tanggal": tanggal,
                "Judul": judul,
                "URL": link,
                "Isi Berita": isi[:300] + "..."  # ringkasan isi
            })
        except Exception as e:
            print(f"❌ Gagal proses artikel: {e}")
            continue

    return hasil

# Jalankan scraping untuk 1 halaman
hasil = scrape_liputan6_page(1)

# Tampilkan hasilnya
for i, item in enumerate(hasil, 1):
    print(f"\n📌 Artikel {i}")
    print(f"Tanggal : {item['Tanggal']}")
    print(f"Judul   : {item['Judul']}")
    print(f"URL     : {item['URL']}")
    print(f"Isi     : {item['Isi Berita']}")
    print("="*80)


🔍 Ditemukan 21 artikel di halaman 1


📄 Memproses artikel: 100%|██████████| 21/21 [00:23<00:00,  1.14s/it]


📌 Artikel 1
Tanggal : 12 Apr 2025 15:05
Judul   : Ini Alasan Pemerintah Kembali Memberlakukan Jurusan IPA, IPS, dan Bahasa di SMA
URL     : https://www.liputan6.com/news/read/5992335/ini-alasan-pemerintah-kembali-memberlakukan-jurusan-ipa-ips-dan-bahasa-di-sma
Isi     : Liputan6.com, Jakarta
Pemerintah dalam hal ini Kementerian Pendidikan Dasar dan Menengah (
Kemendikdasmen
) kembali memberlakukan penjurusan IPA, IPS, dan Bahasa di tingkat Sekolah Menengah Atas (SMA).
Mendikdasmen Abdul Mu'ti menjelaskan, kembali menghidupkan penjurusan
IPA
, IPS, dan Bahasa di tin...

📌 Artikel 2
Tanggal : 04 Mar 2025 12:00
Judul   : Contoh Penilaian Hasil Karya Anak TK Kurikulum Merdeka: Panduan Lengkapnya
URL     : https://www.liputan6.com/feeds/read/5943193/contoh-penilaian-hasil-karya-anak-tk-kurikulum-merdeka-panduan-lengkapnya
Isi     : Liputan6.com, Jakarta
Penilaian hasil karya anak merupakan salah satu komponen penting dalam implementasi Kurikulum Merdeka di tingkat Taman Kanak-kanak (TK). M

In [2]:
import time
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import requests

# Fungsi ambil isi artikel
def ambil_isi_liputan6(url):
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        res = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(res.text, "html.parser")
        konten = soup.find("div", class_="article-content-body__item-content")
        if konten:
            return konten.get_text(separator="\n", strip=True)
        else:
            return "Isi tidak ditemukan"
    except Exception as e:
        return f"Gagal mengambil isi: {e}"

# Scrape per halaman
def scrape_liputan6_page(page):
    url = f"https://www.liputan6.com/tag/kurikulum-merdeka?page={page}"
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    artikel_list = soup.find_all("article", class_="articles--iridescent-list--text-item")
    hasil = []

    for artikel in tqdm(artikel_list, desc=f"📄 Halaman {page}"):
        try:
            a_tag = artikel.find("a", class_="articles--iridescent-list--text-item__title-link")
            if not a_tag:
                continue
            link = a_tag["href"]
            judul = a_tag.get_text(strip=True)
            time_tag = artikel.find("time")
            tanggal = time_tag["title"] if time_tag and "title" in time_tag.attrs else "Tanggal tidak ditemukan"
            isi = ambil_isi_liputan6(link)

            hasil.append({
                "Tanggal": tanggal,
                "Judul": judul,
                "URL": link,
                "Isi Berita": isi
            })
        except Exception as e:
            print(f"❌ Gagal proses artikel: {e}")
            continue

    return hasil

# Looping semua halaman
semua_data = []
for page in range(1, 5):  # halaman 1-4
    data = scrape_liputan6_page(page)
    semua_data.extend(data)

# Simpan ke Excel
df = pd.DataFrame(semua_data)
df.to_excel("liputan6_kurikulum_merdeka.xlsx", index=False)
print("✅ Data berhasil disimpan ke liputan6_kurikulum_merdeka.xlsx")


📄 Halaman 4: 100%|██████████| 17/17 [00:19<00:00,  1.14s/it]


✅ Data berhasil disimpan ke liputan6_kurikulum_merdeka.xlsx


## Metro TV News

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup Selenium
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

def ambil_isi_berita(url):
    driver.get(url)
    time.sleep(2)  # tunggu awal

    # Scroll ke bawah untuk muat isi
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "date"))
        )
        soup = BeautifulSoup(driver.page_source, "html.parser")
        tanggal = soup.find("p", class_="date")
        tanggal = tanggal.get_text(strip=True) if tanggal else "Tanggal tidak ditemukan"

        isi_paragraf = soup.find_all("p")
        isi = " ".join(p.get_text(strip=True) for p in isi_paragraf)
        return tanggal, isi
    except Exception as e:
        return "Tanggal tidak ditemukan", "Isi tidak ditemukan..."

def scrape_metrotv_page(page=1):
    url = f"https://www.metrotvnews.com/search/kurikulum%20merdeka/{page - 1}"
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    items = soup.find_all("div", class_="item")
    
    hasil = []
    for item in items:
        a_tag = item.find("a")
        if not a_tag:
            continue
        judul = a_tag.get("title", "Judul tidak ditemukan")
        href = a_tag["href"]
        link = f"https://www.metrotvnews.com{href}" if not href.startswith("http") else href
        tanggal, isi = ambil_isi_berita(link)

        hasil.append({
            "Judul": judul,
            "Tanggal": tanggal,
            "URL": link,
            "Isi Berita": isi
        })
    
    return hasil

# Test 1 page
# Test 1 page
data = scrape_metrotv_page(1)

# Print hasil lengkap
for item in data:
    print(f"Judul   : {item['Judul']}")
    print(f"Tanggal : {item['Tanggal']}")
    print(f"URL     : {item['URL']}")
    print(f"Isi     : {item['Isi Berita'][:300]}...")  # batasin biar ga kepanjangan
    print("="*80)


Judul   : Kebijakan Pelajaran Bahasa Inggris untuk Usia Dini Dinilai Sudah Tepat
Tanggal : 22 April 2025 19:06
URL     : https://www.metrotvnews.com/play/N6GCx8zd-kebijakan-pelajaran-bahasa-inggris-untuk-usia-dini-dinilai-sudah-tepat
Isi     : ADVERTISEMENT Nasional ADVERTISEMENT 22 April 2025 19:06 SHARE NOW Kementerian Pendidikantelah menetapkan bahwa mulai Tahun Ajaran 2027-2028, bahasa Inggris akan menjadi mata pelajaran wajib di jenjang sekolah dasar. Pemerintah menyerukan pentingnya penguasaan bahasa Inggris sejak dini untuk membant...
Judul   : Kurikulum Merdeka? Bye!
Tanggal : 17 April 2025 13:40
URL     : https://www.metrotvnews.com/play/NleC808d-kurikulum-merdeka-bye
Isi     : ADVERTISEMENT Nasional ADVERTISEMENT 17 April 2025 13:40 SHARE NOW Jakarta:Pemerintah kembali menggulirkan kebijakan sistem penjurusan di tingkat sekolah menengah atas atau SMA.ADVERTISEMENTSetelah sempat dihapus lewat Kurikulum Merdeka, sistem jurusan IPA, IPS, dan Bahasa akan kembali diberlakukan ...


In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup headless browser
options = Options()
options.add_argument("--headless")
driver = webdriver.Chrome(options=options)

def ambil_isi_berita(url):
    driver.get(url)
    time.sleep(2)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "date"))
        )
        soup = BeautifulSoup(driver.page_source, "html.parser")
        tanggal = soup.find("p", class_="date")
        tanggal = tanggal.get_text(strip=True) if tanggal else "Tanggal tidak ditemukan"

        konten = soup.find("div", class_="news-text")
        isi = konten.get_text(separator=" ", strip=True) if konten else "Isi tidak ditemukan..."
        return tanggal, isi
    except Exception as e:
        return "Tanggal tidak ditemukan", "Isi tidak ditemukan..."

def scrape_metrotv_page(page=0):
    url = f"https://www.metrotvnews.com/search/kurikulum%20merdeka/{page}"
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    items = soup.find_all("div", class_="item")
    
    hasil = []
    for item in items:
        a_tag = item.find("a")
        if not a_tag:
            continue
        judul = a_tag.get("title", "Judul tidak ditemukan")
        href = a_tag["href"]
        link = f"https://www.metrotvnews.com{href}" if not href.startswith("http") else href
        tanggal, isi = ambil_isi_berita(link)

        hasil.append({
            "Judul": judul,
            "Tanggal": tanggal,
            "Link": link,
            "Isi Berita": isi
        })
    
    return hasil

# Loop halaman 0 dan 1
semua_data = []
for halaman in [0, 1]:
    hasil = scrape_metrotv_page(halaman)
    semua_data.extend(hasil)

# Simpan ke Excel
df = pd.DataFrame(semua_data)
df.to_excel("MetroTV_Kurikulum Merdeka.xlsx", index=False)

driver.quit()

print("Scraping selesai. Data tersimpam")

Scraping selesai. Data tersimpam


## Kumparan.com

In [ ]:
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm

# Setup driver
options = Options()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(service=Service(), options=options)

# Buka halaman awal Kumparan search
driver.get("https://kumparan.com/tag/kurikulum-merdeka")

# Scroll dan ambil 5 link artikel yang valid
links = []
while len(links) < 5:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    articles = driver.find_elements(By.CSS_SELECTOR, "a[href*='/'][href^='/']")
    for a in articles:
        href = a.get_attribute('href')
        if href and re.match(r'https://kumparan\.com/[^/]+/[^/]+-\w+$', href) and href not in links:
            links.append(href)
        if len(links) == 5:
            break

# Ambil detail artikel
for link in links:
    try:
        driver.get(link)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Judul
        judul_tag = soup.select_one("h1[data-qa-id='story-title']")
        judul = judul_tag.text.strip() if judul_tag else "Judul tidak ditemukan"

        # Tanggal
        tanggal_tag = soup.select_one("span[data-qa-id='story-publish-date']")
        tanggal = tanggal_tag.text.strip() if tanggal_tag else "Tanggal tidak ditemukan"

        # Isi
        isi_el = soup.select("div.TextBoxweb__StyledTextBox-sc-1noultz-0 span[data-qa-id='story-paragraph']")
        isi = "\n".join([p.get_text(separator=" ", strip=True) for p in isi_el])
        isi = isi if isi else "Isi tidak ditemukan"

        # Cetak hasil
        print("="*80)
        print(f"Judul   : {judul}")
        print(f"Tanggal : {tanggal}")
        print(f"Link    : {link}")
        print(f"Isi     : {isi[:300]}..." if len(isi) > 300 else f"Isi     : {isi}")
    except Exception as e:
        print("="*80)
        print(f"Gagal mengambil {link}: {e}")

driver.quit()


🔎 Mengumpulkan Link Artikel: 100%|██████████| 100/100 [00:32<00:00,  3.12it/s]


📰 Scraping Detail Artikel:


📄 Detail: 100%|██████████| 100/100 [05:18<00:00,  3.19s/it]

✅ Data disimpan ke 'kumparan_kurikulum_merdeka.xlsx'


In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup headless browser
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

# Buka halaman pencarian Kumparan
search_url = "https://kumparan.com/search/Kurikulum%20Merdeka"
driver.get(search_url)
time.sleep(5)

# Scroll & ambil 300 link artikel unik
article_links = set()
prev_height = driver.execute_script("return document.body.scrollHeight")

while len(article_links) < 300:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    cards = soup.find_all("div", attrs={"data-qa-id": "story-result-card"})
    for card in cards:
        a = card.find("a", attrs={"aria-label": "news-card-label-link"})
        if not a:
            continue
        title_tag = a.find("span", attrs={"data-qa-id": "title"})
        if title_tag:
            href = a.get("href")
            full_link = "https://kumparan.com" + href
            tanggal_tag = card.find("span", {"data-qa-id": "card-footer-date"})
            tanggal = tanggal_tag.text.strip() if tanggal_tag else "Tanggal tidak ditemukan"
            article_links.add((title_tag.text.strip(), full_link, tanggal))
        if len(article_links) >= 300:
            break

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

print(f"\nTotal artikel ditemukan: {len(article_links)}\n")

# Kumpulkan detail artikel
data_artikel = []
for idx, (judul, link, tanggal) in enumerate(list(article_links), 1):
    print(f"[{idx}/{len(article_links)}] Mengambil: {judul}")
    try:
        driver.get(link)
        time.sleep(4)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        # Ambil tanggal detail
        tanggal_tag = (
            soup.find("time") or
            soup.find("span", {"data-qa-id": "story-date"}) or
            next((span for span in soup.find_all("span") if any(bln in span.text for bln in ["Januari","Februari","Maret","April","Mei","Juni","Juli","Agustus","September","Oktober","November","Desember"])), None)
        )
        tanggal_detail = tanggal_tag.text.strip() if tanggal_tag else tanggal

        # Ambil isi
        content_div = soup.find("div", {"data-qa-id": "story-description"})
        if content_div:
            isi = content_div.get_text(separator=" ", strip=True)
        else:
            isi_paragraf = soup.find_all(["p", "span"], string=True)
            isi = " ".join(el.get_text(strip=True) if hasattr(el, "get_text") else el.strip() for el in isi_paragraf)
            isi = isi if isi else "Isi tidak ditemukan..."

        data_artikel.append({
            "Judul": judul,
            "Link": link,
            "Tanggal": tanggal_detail,
            "Isi": isi
        })
    except Exception as e:
        print(f"   Gagal mengambil artikel: {e}")

# Simpan ke Excel
df = pd.DataFrame(data_artikel)
df.to_excel("Artikel_Kurikulum_Merdeka_Kumparan.xlsx", index=False)
print("\nData berhasil disimpan ke 'Artikel_Kurikulum_Merdeka_Kumparan.xlsx'.")

driver.quit()



Total artikel ditemukan: 300

[1/300] Mengambil: Kebijakan Implementasi Kurikulum Merdeka dan Ciri-Cirinya
[2/300] Mengambil: Materi Fisika Kelas 10 Semester 2 Kurikulum Merdeka
[3/300] Mengambil: Rangkuman Materi IPS Kelas 9 Kurikulum Merdeka
[4/300] Mengambil: 3 Cara Membuat Tujuan Pembelajaran Kurikulum Merdeka
[5/300] Mengambil: Arti PMM dalam Kurikulum Merdeka dan Manfaatnya
[6/300] Mengambil: Materi Matematika Kelas 12 Semester 2 Kurikulum Merdeka
[7/300] Mengambil: 15 Contoh Soal UTS PAI Kelas 1 Semester 2 Kurikulum Merdeka
[8/300] Mengambil: Kunci Jawaban IPS Kelas 8 Halaman 142 Kurikulum Merdeka: Nasionalisme
[9/300] Mengambil: Rangkuman Materi Cerpen Kelas 11 Kurikulum Merdeka
[10/300] Mengambil: 5 Soal PAS Bahasa Indonesia Kelas 7 Semester 2 Kurikulum Merdeka dan Jawabannya
[11/300] Mengambil: Mengenal Contoh Penilaian Sikap Kurikulum Merdeka
[12/300] Mengambil: Prinsip Pembelajaran yang Dirancang dalam Kurikulum Merdeka
[13/300] Mengambil: Salah Satu Perubahan Struktur Kur

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup headless browser
options = Options()
options.add_argument("--disable-gpu")
driver = webdriver.Chrome(options=options)

# Buka halaman pencarian Kumparan
search_url = "https://kumparan.com/search/Kurikulum%20Merdeka"
driver.get(search_url)
time.sleep(5)

# Scroll & ambil 100 link artikel unik
article_links = set()
prev_height = driver.execute_script("return document.body.scrollHeight")

while len(article_links) < 300:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    for a in soup.find_all("a", attrs={"aria-label": "news-card-label-link"}):
        title_tag = a.find("span", attrs={"data-qa-id": "title"})
        if title_tag:
            href = a.get("href")
            full_link = "https://kumparan.com" + href

            # Cari elemen induk buat ambil tanggal
            parent = a.find_parent("div", attrs={"data-qa-id": "story-result-card"})
            date_tag = parent.find("span", {"data-qa-id": "card-footer-date"}) if parent else None
            tanggal = date_tag.text.strip() if date_tag else "Tanggal tidak ditemukan"

            article_links.add((title_tag.text.strip(), full_link, tanggal))
        if len(article_links) >= 300:
            break

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break  # Stop if can't scroll further
    prev_height = new_height

print(f"\nTotal artikel ditemukan: {len(article_links)}\n")

# Ambil detail dari setiap artikel
data = []
for idx, (judul, link, tanggal) in enumerate(list(article_links)[:300], 1):
    print(f"[{idx}/300] Mengambil: {judul}")
    driver.get(link)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Tanggal
    tanggal_tag = soup.find("span", {"data-qa-id": "story-date"})
    if not tanggal_tag:
        span_tags = soup.find_all("span")
        for span in span_tags:
            if span.text.strip() and any(tahun in span.text for tahun in ["2022", "2023", "2024", "2025"]):
                tanggal_tag = span
                break
    tanggal = tanggal_tag.text.strip() if tanggal_tag else "Tanggal tidak ditemukan"

    # Isi berita
    content_div = soup.find("div", {"data-qa-id": "story-description"})
    if content_div:
        isi = content_div.get_text(separator=" ", strip=True)
    else:
        span_paragraphs = soup.find_all("span", {"data-qa-id": "story-paragraph"})
        if span_paragraphs:
            isi = " ".join(span.get_text(strip=True) for span in span_paragraphs)
        else:
            paragraphs = soup.find_all("p")
            isi = " ".join(p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True))
        isi = isi if isi else "Isi tidak ditemukan..."

    data.append({
        "Judul": judul,
        "Tanggal": tanggal,
        "Link": link,
        "Isi Berita": isi
    })

    # Print Progress
    print(f"Progress: {idx}/300 Artikel berhasil diambil")

# Simpan ke Excel
df = pd.DataFrame(data)
df.to_excel("kumparan_kurikulum_merdeka_300.xlsx", index=False)
print("\n✅ Data berhasil disimpan ke 'kumparan_kurikulum_merdeka_100.xlsx'")

driver.quit()


Total artikel ditemukan: 300

[1/300] Mengambil: Kebijakan Implementasi Kurikulum Merdeka dan Ciri-Cirinya
Progress: 1/300 Artikel berhasil diambil
[2/300] Mengambil: Materi Fisika Kelas 10 Semester 2 Kurikulum Merdeka
Progress: 2/300 Artikel berhasil diambil
[3/300] Mengambil: Rangkuman Materi IPS Kelas 9 Kurikulum Merdeka
Progress: 3/300 Artikel berhasil diambil
[4/300] Mengambil: Contoh Modul Ajar Kurikulum Merdeka Semester 2 Jenjang SD
Progress: 4/300 Artikel berhasil diambil
[5/300] Mengambil: 3 Cara Membuat Tujuan Pembelajaran Kurikulum Merdeka
Progress: 5/300 Artikel berhasil diambil
[6/300] Mengambil: Arti PMM dalam Kurikulum Merdeka dan Manfaatnya
Progress: 6/300 Artikel berhasil diambil
[7/300] Mengambil: Materi Matematika Kelas 12 Semester 2 Kurikulum Merdeka
Progress: 7/300 Artikel berhasil diambil
[8/300] Mengambil: 15 Contoh Soal UTS PAI Kelas 1 Semester 2 Kurikulum Merdeka
Progress: 8/300 Artikel berhasil diambil
[9/300] Mengambil: Kunci Jawaban IPS Kelas 8 Halaman 142 

## Tempo

Tempo print dulu

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

driver = webdriver.Chrome()
driver.get("https://www.tempo.co/search?q=kurikulum+merdeka&page=2")
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "figure")))

soup = BeautifulSoup(driver.page_source, 'lxml')
articles = soup.find_all('figure')

count = 0
for article in articles:
    if count >= 5:
        break

    try:
        figcaption = article.find('figcaption')
        if not figcaption:
            continue

        a_tag = figcaption.find('a')
        if not a_tag:
            continue

        title = a_tag.text.strip()
        link = "https://www.tempo.co" + a_tag['href']

        # buka link berita
        driver.get(link)
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        time.sleep(2)

        article_soup = BeautifulSoup(driver.page_source, 'lxml')

        # Ambil tanggal dari <p class="text-neutral-900 text-sm">
        tanggal_tag = article_soup.find('p', class_='text-neutral-900 text-sm')
        date = tanggal_tag.text.strip() if tanggal_tag else 'Tanggal tidak ditemukan'

        # Coba beberapa alternatif lokasi isi konten
        content_div = (
            article_soup.find('div', id='isi') or
            article_soup.find('div', class_='article-content') or
            article_soup.find('article')
        )

        if content_div:
            paragraphs = content_div.find_all('p')
            content = '\n'.join(p.text for p in paragraphs if p.text.strip())
        else:
            content = 'Konten tidak ditemukan'

        # Cetak hasil
        count += 1
        print(f"{count}. Judul   : {title}")
        print(f"   Tanggal : {date}")
        print(f"   Link    : {link}")
        print(f"   Isi     : {content[:300]}...\n{'='*80}\n")

    except Exception as e:
        print("Gagal mengambil data:", e)

driver.quit()


1. Judul   : Pro Kontra atas Rencana Penerapan Kembali Sistem Penjurusan di SMA
   Tanggal : 15 April 2025 | 19.08 WIB
   Link    : https://www.tempo.co/politik/pro-kontra-atas-rencana-penerapan-kembali-sistem-penjurusan-di-sma--1231666
   Isi     :  Baca berita dengan sedikit iklan,  klik disini 
RENCANA pemerintah menerapkan kembali sistem penjurusan untuk jenjang pendidikan sekolah menengah atas (SMA) pada tahun ajaran 2025/2026 mendapat sorotan dari berbagai kalangan. Sistem itu telah dihapus pada tahun ajaran 2024/2025 dan Menteri Pendidik...

Gagal mengambil data: Incoming markup is of an invalid type: None. Markup must be a string, a bytestring, or an open filehandle.
Gagal mengambil data: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=135.0.7049.116)
Stacktrace:
	GetHandleVerifier [0x00007FF6FF3DEFA5+77893]
	GetHandleVerifier [0x00007FF6FF3DF000+77984]
	(No symbol) [0x00007FF6FF1A91BA]
	(No symbol) [0x00007FF

Tempo ke excel

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome()
data = []

# Loop dari halaman 1 sampai 18
for page in range(1, 19):
    print(f"\n🔎 Memproses halaman {page}...\n")
    driver.get(f"https://www.tempo.co/search?q=kurikulum+merdeka&page={page}")
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.TAG_NAME, "figure")))

    soup = BeautifulSoup(driver.page_source, 'lxml')
    articles = soup.find_all('figure')

    for article in articles:
        try:
            figcaption = article.find('figcaption')
            if not figcaption:
                continue

            a_tag = figcaption.find('a')
            if not a_tag:
                continue

            title = a_tag.text.strip()
            link = "https://www.tempo.co" + a_tag['href']

            # Buka artikel detail
            driver.get(link)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            time.sleep(2)

            article_soup = BeautifulSoup(driver.page_source, 'lxml')

            # Ambil tanggal dari <p class="text-neutral-900 text-sm">
            tanggal_tag = article_soup.find('p', class_='text-neutral-900 text-sm')
            date = tanggal_tag.text.strip() if tanggal_tag else 'Tanggal tidak ditemukan'

            # Ambil isi berita
            content_div = (
                article_soup.find('div', id='isi') or
                article_soup.find('div', class_='article-content') or
                article_soup.find('article')
            )

            if content_div:
                paragraphs = content_div.find_all('p')
                content = '\n'.join(p.text for p in paragraphs if p.text.strip())
            else:
                content = 'Konten tidak ditemukan'

            # Simpan ke list
            data.append({
                "Judul": title,
                "Tanggal": date,
                "Link": link,
                "Isi Berita": content
            })

            print(f"✅ {title}")

        except Exception as e:
            print(f"⚠️ Gagal mengambil artikel: {e}")

# Simpan ke Excel
df = pd.DataFrame(data)
df.to_excel("tempo_kurikulum_merdeka.xlsx", index=False)
print("\n✅ Data berhasil disimpan ke 'tempo_kurikulum_merdeka.xlsx'")

driver.quit()


🔎 Memproses halaman 1...

✅ Pro Kontra atas Rencana Penerapan Kembali Sistem Penjurusan di SMA
✅ Menteri Abdul Mu'ti Diminta Evaluasi Kurikulum Merdeka Sebelum Menerapkan Sistem Penjurusan SMA
✅ Aktivis Pendidikan Beberkan Keunggulan Sistem Penjurusan di SMA
✅ Nadiem Makarim Hapus Sistem Penjurusan di SMA, Abdul Mu'ti Hidupkan Kembali
✅ Menteri Abdul Mu'ti Kembali Terapkan Sistem Penjurusan di SMA
✅ Menyesatkan: Klaim Ideologi Sekuler Sosialisme dan PKI Dalam Kurikulum Merdeka Belajar
✅ Sumatera Selatan Uji Coba Kurikulum Pangan Lokal di 34 SMA/SMK
✅ Pangan Lokal untuk Ketahanan Iklim Masuk Kurikulum SMA dan SMK, Dimulai dari Daerah Ini
✅ Mahasiswa KKN Unair Ajari Guru Susun Modul Kurikulum Merdeka Pakai AI
✅ Apa itu Kurikulum? Ini Tujuan, Fungsi, Kedudukan, dan Contohnya

🔎 Memproses halaman 2...

✅ Pro Kontra atas Rencana Penerapan Kembali Sistem Penjurusan di SMA
✅ Menteri Abdul Mu'ti Diminta Evaluasi Kurikulum Merdeka Sebelum Menerapkan Sistem Penjurusan SMA
⚠️ Gagal mengambil art

InvalidSessionIdException: Message: invalid session id
Stacktrace:
	GetHandleVerifier [0x00007FF6FF3DEFA5+77893]
	GetHandleVerifier [0x00007FF6FF3DF000+77984]
	(No symbol) [0x00007FF6FF1A8FEC]
	(No symbol) [0x00007FF6FF1EF44F]
	(No symbol) [0x00007FF6FF226FF2]
	(No symbol) [0x00007FF6FF221A02]
	(No symbol) [0x00007FF6FF220AC9]
	(No symbol) [0x00007FF6FF175AB5]
	GetHandleVerifier [0x00007FF6FF697B6D+2931725]
	GetHandleVerifier [0x00007FF6FF692132+2908626]
	GetHandleVerifier [0x00007FF6FF6B00F3+3031443]
	GetHandleVerifier [0x00007FF6FF3F91EA+184970]
	GetHandleVerifier [0x00007FF6FF40086F+215311]
	(No symbol) [0x00007FF6FF174B2C]
	GetHandleVerifier [0x00007FF6FF7A2518+4023736]
	BaseThreadInitThunk [0x00007FFEA0D9E8D7+23]
	RtlUserThreadStart [0x00007FFEA26F14FC+44]


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)
data = []

driver.get("https://www.tempo.co/search?q=kurikulum+merdeka")

# Loop halaman dari 1 sampai 18
for page in range(1, 19):
    print(f"\n🔎 Memproses halaman {page}...\n")
    
    # Tunggu sampai tombol-tombol pagination muncul
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "button[data-type='page']")))

    # Klik tombol sesuai nomor halaman
    if page > 1:
        try:
            page_buttons = driver.find_elements(By.CSS_SELECTOR, "button[data-type='page']")
            for btn in page_buttons:
                if btn.text.strip() == str(page):
                    # Scroll tombol ke tengah layar agar tidak tertutup header
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", btn)
                    time.sleep(0.5)
                    # Klik pakai JavaScript untuk menghindari intercept
                    driver.execute_script("arguments[0].click();", btn)
                    break
            time.sleep(2)  # Tunggu isi halaman dimuat
        except Exception as e:
            print(f"⚠️ Gagal klik pagination halaman {page}: {e}")
            continue

    wait.until(EC.presence_of_all_elements_located((By.TAG_NAME, "figure")))
    soup = BeautifulSoup(driver.page_source, 'lxml')
    articles = soup.find_all('figure')

    for article in articles:
        try:
            figcaption = article.find('figcaption')
            if not figcaption:
                continue

            a_tag = figcaption.find('a')
            if not a_tag:
                continue

            title = a_tag.text.strip()
            link = "https://www.tempo.co" + a_tag['href']

            # Buka artikel detail
            driver.get(link)
            wait.until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
            time.sleep(2)

            article_soup = BeautifulSoup(driver.page_source, 'lxml')
            tanggal_tag = article_soup.find('p', class_='text-neutral-900 text-sm')
            date = tanggal_tag.text.strip() if tanggal_tag else 'Tanggal tidak ditemukan'

            content_div = (
                article_soup.find('div', id='isi') or
                article_soup.find('div', class_='article-content') or
                article_soup.find('article')
            )

            if content_div:
                paragraphs = content_div.find_all('p')
                content = '\n'.join(p.text for p in paragraphs if p.text.strip())
            else:
                content = 'Konten tidak ditemukan'

            data.append({
                "Judul": title,
                "Tanggal": date,
                "Link": link,
                "Isi Berita": content
            })

            print(f"✅ {title}")
            driver.back()  # Kembali ke halaman sebelumnya
            time.sleep(2)

        except Exception as e:
            print(f"⚠️ Gagal mengambil artikel: {e}")
            driver.get("https://www.tempo.co/search?q=kurikulum+merdeka")
            time.sleep(2)

df = pd.DataFrame(data)
df.to_excel("tempo_kurikulum_merdeka.xlsx", index=False)
print("\n✅ Data berhasil disimpan ke 'tempo_kurikulum_merdeka.xlsx'")

driver.quit()



🔎 Memproses halaman 1...

✅ Pro Kontra atas Rencana Penerapan Kembali Sistem Penjurusan di SMA
✅ Menteri Abdul Mu'ti Diminta Evaluasi Kurikulum Merdeka Sebelum Menerapkan Sistem Penjurusan SMA
✅ Aktivis Pendidikan Beberkan Keunggulan Sistem Penjurusan di SMA
✅ Nadiem Makarim Hapus Sistem Penjurusan di SMA, Abdul Mu'ti Hidupkan Kembali
✅ Menteri Abdul Mu'ti Kembali Terapkan Sistem Penjurusan di SMA
✅ Menyesatkan: Klaim Ideologi Sekuler Sosialisme dan PKI Dalam Kurikulum Merdeka Belajar
✅ Sumatera Selatan Uji Coba Kurikulum Pangan Lokal di 34 SMA/SMK
✅ Pangan Lokal untuk Ketahanan Iklim Masuk Kurikulum SMA dan SMK, Dimulai dari Daerah Ini
✅ Mahasiswa KKN Unair Ajari Guru Susun Modul Kurikulum Merdeka Pakai AI
✅ Apa itu Kurikulum? Ini Tujuan, Fungsi, Kedudukan, dan Contohnya

🔎 Memproses halaman 2...

✅ Mendikdasmen Abdul Mu'ti: Pendekatan Deep Learning Akan Diterapkan di Kurikulum Nasional
✅ Deretan Kurikulum yang Pernah Diterapkan di Indonesia Dalam 20 Tahun Terakhir
✅ Mendikdasmen Teg

: 

## Viva

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()
driver.get("https://www.viva.co.id/tag/kurikulum-merdeka")
time.sleep(3)

# Ambil 1 link pertama
first_link = driver.find_element(By.CSS_SELECTOR, "div.article-list-info a.article-list-title")
href = first_link.get_attribute("href")

print("URL diambil:", href)

# Kunjungi langsung URL-nya
driver.get(href)
time.sleep(5)

print("Current URL (after redirect):", driver.current_url)
print("Page source contains <article>? -->", "<article" in driver.page_source)

driver.quit()


URL diambil: https://www.viva.co.id/edukasi/1763379-satu-dekade-pembangunan-pendidikan-semakin-berdampak-dan-bermanfaat
Current URL (after redirect): https://www.viva.co.id/edukasi/1763379-satu-dekade-pembangunan-pendidikan-semakin-berdampak-dan-bermanfaat
Page source contains <article>? --> False


: 

## Merdeka

In [6]:
import requests
from bs4 import BeautifulSoup

def scrape_merdeka_article_detail(url):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")

    # Ambil judul
    title_tag = soup.find("h1", class_="article-title")
    title = title_tag.get_text(strip=True) if title_tag else ""

    # Ambil tanggal
    time_tag = soup.find("time")
    date = time_tag.get_text(strip=True) if time_tag else ""

    # Ambil isi artikel
    article_div = soup.find("div", class_="article article--version2 relative pb-6")
    paragraphs = article_div.find_all(["p", "h2", "h3"]) if article_div else []
    content = "\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])

    return {
        "url": url,
        "title": title,
        "date": date,
        "content": content
    }

# Contoh pemanggilan fungsi
url = "https://www.merdeka.com/jateng/kurikulum-merdeka-adalah-kurikulum-baru-di-indonesia-ketahui-sistemnya-62980-mvk.html"
article_data = scrape_merdeka_article_detail(url)
print(article_data)


{'url': 'https://www.merdeka.com/jateng/kurikulum-merdeka-adalah-kurikulum-baru-di-indonesia-ketahui-sistemnya-62980-mvk.html', 'title': 'Kurikulum Merdeka adalah Kurikulum Baru di Indonesia, Ketahui Sistemnya', 'date': 'Jumat, 15 Des 202320:38:00', 'content': 'Kurikulum Merdeka berfokus pada pembelajaran sesuai kebutuhan minat dan bakat anak.\nKurikulum Merdeka berfokus pada pembelajaran sesuai kebutuhan minat dan bakat anak.\nKurikulum Merdeka adalah Kurikulum Baru di Indonesia, Ketahui Sistemnya\nDalam hal ini, penting bagi masyarakat untuk memahami dengan baik apa itu Kurikulum Merdeka. Selain pengertian, Anda juga perlu mengetahui tujuan Kurikulum Merdeka, karakteristik, dan prinsip pembelajaran. Bukan hanya itu, perlu juga dipahami bagaimana tahap pelaksanaan Kurikulum Merdeka yang kini diterapkan.Tidak hanya pelajar dan mahasiswa yang harus memahami konsep Kurikulum Merdeka, orang tua pun perlu memahaminya. Hal ini tidak lain untuk mendukung proses belajar anak. Kami merangkum b

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Konfigurasi headless browser
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--no-sandbox")

# Jalankan driver
driver = webdriver.Chrome(options=options)

# Buka halaman pencarian
search_url = "https://www.merdeka.com/search?q=kurikulum+merdeka"
driver.get(search_url)
time.sleep(3)  # tunggu render

# Ambil source code dan parse pakai BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

# Cari semua link hasil pencarian
results = soup.find_all("div", class_="gs-title")

articles = []

for result in results:
    a_tag = result.find("a", class_="gs-title")
    if a_tag and a_tag.get("href"):
        title = a_tag.get_text(strip=True)
        link = a_tag["href"]

        # Kunjungi halaman detail berita
        driver = webdriver.Chrome(options=options)
        driver.get(link)
        time.sleep(3)
        detail_soup = BeautifulSoup(driver.page_source, "html.parser")
        driver.quit()

        # Isi artikel
        article_div = detail_soup.find("div", class_="article article--version2 relative pb-6")
        paragraphs = article_div.find_all(["p", "h2", "h3"]) if article_div else []
        article_text = "\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])

        # Tanggal
        time_tag = detail_soup.find("time")
        date = time_tag.get_text(strip=True) if time_tag else "Tanggal tidak ditemukan"

        # Simpan
        articles.append({
            "judul": title,
            "link": link,
            "tanggal": date,
            "isi": article_text
        })

# Cetak hasil
for i, art in enumerate(articles, 1):
    print(f"\nBerita #{i}")
    print("Judul:", art['judul'])
    print("Tanggal:", art['tanggal'])
    print("Link:", art['link'])
    print("Isi (potongan):", art['isi'][:300], "...\n")



Berita #1
Judul: Kurikulum Merdekaadalah Kurikulum Baru di Indonesia, Ketahui ...
Tanggal: Jumat, 15 Des 202320:38:00
Link: https://www.merdeka.com/jateng/kurikulum-merdeka-adalah-kurikulum-baru-di-indonesia-ketahui-sistemnya-62980-mvk.html
Isi (potongan): Kurikulum Merdeka berfokus pada pembelajaran sesuai kebutuhan minat dan bakat anak.
Kurikulum Merdeka berfokus pada pembelajaran sesuai kebutuhan minat dan bakat anak.
Kurikulum Merdeka adalah Kurikulum Baru di Indonesia, Ketahui Sistemnya
Dalam hal ini, penting bagi masyarakat untuk memahami dengan ...


Berita #2
Judul: Kurikulum Merdekaadalah Kurikulum Baru di Indonesia, Ketahui ...
Tanggal: Jumat, 15 Des 202320:38:00
Link: https://www.merdeka.com/jateng/kurikulum-merdeka-adalah-kurikulum-baru-di-indonesia-ketahui-sistemnya-62980-mvk.html
Isi (potongan): Kurikulum Merdeka berfokus pada pembelajaran sesuai kebutuhan minat dan bakat anak.
Kurikulum Merdeka berfokus pada pembelajaran sesuai kebutuhan minat dan bakat anak.
Kurikulum 

In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from tqdm import tqdm

# Setup Selenium Headless
options = Options()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(options=options)

# Buka halaman awal pencarian
search_url = "https://www.merdeka.com/search?q=kurikulum+merdeka"
driver.get(search_url)
time.sleep(3)

articles = []

# Loop dari halaman 1 sampai 10
for page in range(1, 11):
    print(f"\nMengambil halaman {page}...")
    time.sleep(2)  # Tunggu render

    # Ambil source code dan parsing
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Cari semua link hasil pencarian
    results = soup.find_all("div", class_="gs-title")

    for result in tqdm(results, desc=f"Scraping Artikel di Halaman {page}", unit="artikel"):
        a_tag = result.find("a", class_="gs-title")
        if a_tag and a_tag.get("href"):
            title = a_tag.get_text(strip=True)
            link = a_tag["href"]

            try:
                # Buka halaman detail berita
                detail_driver = webdriver.Chrome(options=options)
                detail_driver.get(link)
                time.sleep(3)
                detail_soup = BeautifulSoup(detail_driver.page_source, "html.parser")
                detail_driver.quit()

                # Isi artikel
                article_div = detail_soup.find("div", class_="article article--version2 relative pb-6")
                paragraphs = article_div.find_all(["p", "h2", "h3"]) if article_div else []
                article_text = "\n".join([p.get_text(strip=True) for p in paragraphs if p.get_text(strip=True)])

                # Tanggal
                time_tag = detail_soup.find("time")
                date = time_tag.get_text(strip=True) if time_tag else "Tanggal tidak ditemukan"

                # Simpan
                articles.append({
                    "judul": title,
                    "link": link,
                    "tanggal": date,
                    "isi": article_text
                })

            except Exception as e:
                print(f"Error scraping artikel: {e}")
                continue

    # Kalau masih ada halaman selanjutnya, klik tombol halaman berikutnya
    # Cari tombol halaman berikutnya
    try:
        next_page_btn = driver.find_element(By.XPATH, f"//div[@class='gsc-cursor-page' and text()='{page+1}']")
        driver.execute_script("arguments[0].scrollIntoView(true);", next_page_btn)  # SCROLL ke tombol
        time.sleep(1)
        driver.execute_script("arguments[0].click();", next_page_btn)  # KLIK pakai JavaScript
        time.sleep(3)  # Tunggu halaman pindah
    except Exception as e:
        print(f"Tidak bisa klik ke halaman {page+1}: {e}")
        break


# Selesai scraping, tutup driver utama
driver.quit()

# Simpan ke Excel
df = pd.DataFrame(articles)
df.to_excel("artikel_kurikulum_merdeka_merdeka_com.xlsx", index=False, engine='openpyxl')

print("\nScraping selesai! Data sudah disimpan di 'artikel_kurikulum_merdeka_merdeka_com.xlsx'.")



Mengambil halaman 1...


Scraping Artikel di Halaman 1: 100%|██████████| 23/23 [05:19<00:00, 13.87s/artikel]



Mengambil halaman 2...


Scraping Artikel di Halaman 2: 100%|██████████| 23/23 [05:39<00:00, 14.76s/artikel]



Mengambil halaman 3...


...


Mengambil halaman 9...


Scraping Artikel di Halaman 9: 100%|██████████| 23/23 [03:27<00:00,  9.01s/artikel]



Mengambil halaman 10...


Scraping Artikel di Halaman 10: 100%|██████████| 23/23 [03:29<00:00,  9.13s/artikel]


Tidak bisa klik ke halaman 11: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='gsc-cursor-page' and text()='11']"}
  (Session info: chrome=135.0.7049.115); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6301BEFA5+77893]
	GetHandleVerifier [0x00007FF6301BF000+77984]
	(No symbol) [0x00007FF62FF891BA]
	(No symbol) [0x00007FF62FFDF16D]
	(No symbol) [0x00007FF62FFDF41C]
	(No symbol) [0x00007FF630032237]
	(No symbol) [0x00007FF63000716F]
	(No symbol) [0x00007FF63002F07F]
	(No symbol) [0x00007FF630006F03]
	(No symbol) [0x00007FF62FFD0328]
	(No symbol) [0x00007FF62FFD1093]
	GetHandleVerifier [0x00007FF630477B6D+2931725]
	GetHandleVerifier [0x00007FF630472132+2908626]
	GetHandleVerifier [0x00007FF6304900F3+3031443]
	GetHandleVerifier [0x00007FF6301D91EA+184970]
	GetHandleVerifier [0x00007FF6301E086F+215311]
	Ge

## SindoNews

In [5]:
import requests
from bs4 import BeautifulSoup

# URL search
url = 'https://search.sindonews.com/go?type=artikel&q=kurikulum+merdeka'

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

# Request halaman search
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# Cari semua elemen <li> (karena di dalam li semua datanya)
articles = soup.find_all('li')

for article in articles:
    title_tag = article.find('div', class_='news-title edukasi-title')
    if title_tag:  # Pastikan ini berita, bukan li lain
        title_link = title_tag.find('a')
        title = title_link.text.strip()
        link = title_link['href']

        # Tanggal ambil dari class 'news-date' di dalam li
        date_tag = article.find('div', class_='news-date')
        published_date = date_tag.text.strip() if date_tag else 'Tanggal tidak ditemukan'

        # Request halaman detail untuk ambil isi berita
        detail_response = requests.get(link, headers=headers)
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

        content_div = detail_soup.find('div', id='detail-desc')
        content = content_div.get_text(separator=' ', strip=True) if content_div else 'Isi tidak ditemukan'

        print(f"Judul: {title}")
        print(f"Tanggal: {published_date}")
        print(f"Link: {link}")
        print(f"Isi: {content}")
        print("-" * 100)


Judul: Mendikdasmen: Kurikulum 2013 dan Kurikulum Merdeka Masih Berlaku
Tanggal: Selasa, 31 Desember 2024 - 17:02 WIB
Link: https://edukasi.sindonews.com/read/1510545/212/mendikdasmen-kurikulum-2013-dan-kurikulum-merdeka-masih-berlaku-1735636321
Isi: JAKARTA - Menteri Pendidikan Dasar dan Menengah (Mendikdasmen) Abdul Mu'ti menegaskan bahwa Kurikulum 2013 dan Kurikulum Merdeka kini masih diberlakukan di satuan pendidikan. Mu'ti menegaskan pihaknya belum melakukan perubahan terhadap dua kurikulum itu. "Sekarang ini masih berlaku dua kurikulum nasional, yaitu Kurikulum 2013 dan Kurikulum Merdeka. Kedua kurikulum itu sekarang masih tetap berlaku dan tidak kita lakukan perubahan," kata Mu'ti di Kantor Kemendikdasmen, Jakarta, Selasa (31/12/2024). Menurut Mu'ti, perubahan itu akan terjadi ketika sudah benar-benar ada perubahan. "Artinya, sebelum ada perubahan maka dua kurikulum itu secara paralel masih berlaku," ujarnya. Baca Juga : Problematika Penerapan Kurikulum Merdeka Namun, kata Mu'ti

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Semua URL yang mau di-scrape
urls = [
    'https://search.sindonews.com/go?type=artikel&q=kurikulum+merdeka',
    'https://search.sindonews.com/go/20?type=artikel&q=kurikulum+merdeka',
    'https://search.sindonews.com/go/40?type=artikel&q=kurikulum+merdeka',
    'https://search.sindonews.com/go/60?type=artikel&q=kurikulum+merdeka'
]

# Header agar tidak ditolak
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

# List buat simpan semua hasil
all_data = []

# Loop semua URL
for url in urls:
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = soup.find_all('li')

    for article in articles:
        title_tag = article.find('div', class_='news-title edukasi-title')
        if title_tag:
            title_link = title_tag.find('a')
            title = title_link.text.strip()
            link = title_link['href']

            # Tanggal dari halaman list
            date_tag = article.find('div', class_='news-date')
            published_date = date_tag.text.strip() if date_tag else 'Tanggal tidak ditemukan'

            # Request halaman detail
            detail_response = requests.get(link, headers=headers)
            detail_soup = BeautifulSoup(detail_response.text, 'html.parser')

            content_div = detail_soup.find('div', id='detail-desc')
            content = content_div.get_text(separator=' ', strip=True) if content_div else 'Isi tidak ditemukan'

            # Simpan ke list
            all_data.append({
                'Judul': title,
                'Tanggal': published_date,
                'Link': link,
                'Isi': content
            })

# Masukkan ke DataFrame
df = pd.DataFrame(all_data)

# Save ke Excel
df.to_excel('Sindonews_Kurikulum Merdeka.xlsx', index=False)

print("Scraping selesai! Data sudah disimpan ke 'Sindonews_Kurikulum Merdeka.xlsx'")


Scraping selesai! Data sudah disimpan ke 'Sindonews_Kurikulum Merdeka.xlsx'


## Jawapos Online

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# Setup Chrome Driver
service = Service()
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=service, options=options)

# Buka halaman search
url = 'https://www.jawapos.com/search?q=kurikulum%20merdeka&sort=latest&page=1'
driver.get(url)
time.sleep(3)  # Tunggu loading JavaScript selesai

# Ambil source HTML setelah render
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Cari semua berita
articles = soup.find_all('div', class_='latest__item')

for article in articles:
    title_tag = article.find('h2', class_='latest__title')
    if title_tag:
        title_link = title_tag.find('a')
        title = title_link.text.strip()
        link = title_link['href']

        # Tanggal
        date_tag = article.find('date', class_='latest__date')
        published_date = date_tag.text.strip() if date_tag else 'Tanggal tidak ditemukan'

        # Buka halaman detail
        driver.get(link)
        time.sleep(2)

        detail_html = driver.page_source
        detail_soup = BeautifulSoup(detail_html, 'html.parser')

        paragraphs = detail_soup.find_all('p')
        content = ' '.join([p.get_text(strip=True) for p in paragraphs])

        print(f"Judul: {title}")
        print(f"Tanggal: {published_date}")
        print(f"Link: {link}")
        print(f"Isi: {content}")
        print("-" * 100)

driver.quit()


Judul: Mendikdasmen Abdul Mu’ti Tegaskan Deep Learning Bukan Pengganti Kurikulum Merdeka
Tanggal: Selasa, 18 Februari 2025 | 07:09 WIB
Link: https://www.jawapos.com/kabinet-merah-putih/015658580/mendikdasmen-abdul-muti-tegaskandeep-learning-bukan-pengganti-kurikulum-merdeka
Isi: ADVERTISEMENT   JawaPos.com- Menteri Pendidikan Dasar dan Menengah (Mendikdasmen) Abdul Mu’ti meluruskan miskonsepsi di masyarakat. Yakni, soal konsepdeep learningdalam pembelajaran. Dia mengatakan,deep learningbukanlah kurikulum baru di era pemerintahannya.Deep learningadalah pendekatan pembelajaran. Di mana, fokusnya menekankan pada pembelajaranmindful, meaningful, danjoyful.Jadi, konsep ini bukanlah sebuah kurikulum baru untuk menggantikan yang lama. “Apalagi dikatakan mengganti labelKurikulum Merdeka. Itu dibanding-bandingkan begitu, padahal ini tidak mengganti label kurikulum yang suda ada,” ujarnya dalam Seminar Nasional dan Sosialisasi Program Deep Learning di UHAMKA Jakarta, Senin (17/2). Baca Juga:Ment

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup Chrome Driver
service = Service()
options = webdriver.ChromeOptions()
# options.add_argument('--headless')  # jangan headless dulu kalau mau aman
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=service, options=options)

# Data kosong
data = []

# Loop dari halaman 1 sampai 8
for page in range(1, 9):
    print(f"Scraping halaman {page}...")

    url = f'https://www.jawapos.com/search?q=kurikulum%20merdeka&sort=latest&page={page}'
    driver.get(url)
    time.sleep(2)

    # Scroll sampai bawah supaya semua berita muncul
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(3)

    # Ambil HTML
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    # Cari semua daftar artikel
    articles = soup.find_all('div', class_='latest__item')

    if not articles:
        print(f"Tidak ada artikel di halaman {page}.")
        continue

    for article in articles:
        title_tag = article.find('h2', class_='latest__title')
        if title_tag:
            title_link = title_tag.find('a')
            title = title_link.text.strip()
            link = title_link['href']

            # Ambil tanggal
            date_tag = article.find('date', class_='latest__date')
            published_date = date_tag.text.strip() if date_tag else 'Tanggal tidak ditemukan'

            # Buka halaman berita
            driver.get(link)
            time.sleep(2)

            detail_html = driver.page_source
            detail_soup = BeautifulSoup(detail_html, 'html.parser')

            # Ambil semua <p>, <div dir="auto">, dan <article>
            paragraphs = detail_soup.find_all('p')
            divs_auto = detail_soup.find_all('div', attrs={"dir": "auto"})
            articles_content = detail_soup.find_all('article', class_='read__content clearfix')

            # Gabungkan semua teks
            content_parts = []

            for p in paragraphs:
                content_parts.append(p.get_text(strip=True))

            for div in divs_auto:
                content_parts.append(div.get_text(strip=True))

            for article in articles_content:
                content_parts.append(article.get_text(strip=True))

            # Gabungkan semua jadi satu isi berita
            content = ' '.join(content_parts)

            data.append({
                'Judul': title,
                'Tanggal': published_date,
                'Link': link,
                'Isi': content
            })

# Tutup browser
driver.quit()

# Simpan ke Excel
df = pd.DataFrame(data)
df.to_excel('jawapos_kurikulum_merdeka.xlsx', index=False)

print("Scraping selesai! Data disimpan ke 'jawapos_kurikulum_merdeka.xlsx'")


Scraping halaman 1...
Scraping halaman 2...
Scraping halaman 3...
Scraping halaman 4...
Scraping halaman 5...
Scraping halaman 6...
Scraping halaman 7...
Scraping halaman 8...
Scraping selesai! Data disimpan ke 'jawapos_kurikulum_merdeka.xlsx'


## IDNTimes

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# Konfigurasi Chrome
options = Options()
# options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")
options.add_argument("--window-size=1920,1080")

driver = webdriver.Chrome(options=options)

# Buka halaman search
url = "https://www.idntimes.com/search?keyword=kurikulum+merdeka"
driver.get(url)

# Scroll sampai mentok
last_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Setelah semua artikel ke-load
soup = BeautifulSoup(driver.page_source, "html.parser")
articles = soup.find_all("li", class_="box-latest box-list")

print(f"Total artikel ditemukan: {len(articles)}\n")

data_artikel = []

# Ambil info satu per satu
for i, article in enumerate(articles, 1):
    try:
        # Judul
        title_tag = article.find("h2", class_="title-text")
        title = title_tag.get_text(strip=True) if title_tag else "Judul tidak ditemukan"

        # Link
        link_tag = article.find("a", class_="box-panel clearfix")
        link = link_tag["href"] if link_tag and link_tag.has_attr("href") else None

        # Tanggal
        time_tag = article.find("time", class_="date")
        date = time_tag.get_text(strip=True) if time_tag else "Tanggal tidak ditemukan"

        # Kalau link ada, buka halaman artikel
        if link:
            driver.get(link)
            time.sleep(2)
            detail_soup = BeautifulSoup(driver.page_source, "html.parser")

            # Cari semua div split-page
            split_pages = detail_soup.find_all("div", class_="split-page")
            paragraphs = []

            for page in split_pages:
                ps = page.find_all(["p", "h2", "h3"])
                paragraphs += [p.get_text(strip=True) for p in ps if p.get_text(strip=True)]

            article_text = "\n".join(paragraphs)

            print(f"Artikel #{i}")
            print(f"Judul: {title}")
            print(f"Tanggal: {date}")
            print(f"Link: {link}")
            print(f"Isi (potongan): {article_text[:300]}...\n")

            # Simpan data
            data_artikel.append({
                "judul": title,
                "tanggal": date,
                "link": link,
                "isi": article_text
            })

    except Exception as e:
        print(f"Error di artikel #{i}: {e}")

driver.quit()


Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)


Total artikel ditemukan: 59

Artikel #1
Judul: Kurikulum Merdeka atau Penjurusan? Sama Saja Bagi Siswa SMA Semarang
Tanggal: 19 Apr 2025
Link: https://jateng.idntimes.com/news/jateng/anggun-puspitoningrum-1/kurikulum-merdeka-atau-penjurusan-sama-saja-bagi-siswa-sma-semarang
Isi (potongan): 1. Belajar lebih terarah dengan sistem penjurusan
Seperti siswi SMA 3 Semarang, Peqsya Jibril Riyadi yang mengaku setuju dengan sistem peminatan jurusan IPA, IPS, atau Bahasa.
‘’Kalau aku sih lebih belajar dengan sistem penjurusan ya, karena belajarnya akan lebih terarah dan bermanfaat untuk menentu...

Artikel #2
Judul: Contoh Soal Bahasa Indonesia Kelas 4 Kurikulum Merdeka
Tanggal: 14 Mar 2025
Link: https://idntimes.com/life/education/ana-widiawati-1/soal-bahasa-indonesia-kelas-4-kurikulum-merdeka
Isi (potongan): 1. Contoh soal Bahasa Indonesia kelas 4 kurikulum Merdeka
1. Rambu di bawah ini berisi petunjuk...
A. SituasiB. KondisiC. ArahD. Peringatan
Jawaban: C
2. Kalimat dengan awalan ber- yang te

In [19]:
import pandas as pd

df = pd.DataFrame(data_artikel)
df.to_excel("IDNTimes_Kurikulum Merdeka.xlsx", index=False)

## UMKM

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup ChromeDriver
service = Service("D:/Tools/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 15)

# Login manual
driver.get("https://satudata.umkm.go.id/login")
input("➡️ Login dan isi captcha di browser, lalu tekan ENTER...")

# Akses halaman filter
driver.get("https://satudata.umkm.go.id/admin/pl/umkm")
time.sleep(3)

# PILIH TABEL
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#select2-jenis_tabel-container"))).click()
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field"))).send_keys("Tabel 1. Usaha Menurut Skala Usaha", Keys.ENTER)
time.sleep(1)

# PILIH MENURUT KEGIATAN USAHA
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='tipe_tabel_kegiatan']"))).click()
time.sleep(1)

# PILIH KATEGORI USAHA
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#select2-jenis_kegiatan-container"))).click()
search_input = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field")))
search_input.send_keys("I.")
search_input.send_keys(Keys.ENTER)
time.sleep(1)

# PILIH KABUPATEN
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#select2-alamat_id_kabkot-container"))).click()
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field"))).send_keys("KAB. JEMBER", Keys.ENTER)
time.sleep(1)

# PILIH KECAMATAN
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#select2-alamat_id_kec-container"))).click()
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field"))).send_keys("35.09.09 BANGSALSARI", Keys.ENTER)
time.sleep(1)

# CEKLIST PL KUMKM
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='tipe_data_pl']"))).click()
time.sleep(1)

# KLIK FILTER
wait.until(EC.element_to_be_clickable((By.ID, "filter"))).click()
time.sleep(5)

# CARI LINK BUTTON PADA KOLOM MIKRO UNTUK BARIS "INFORMASI DAN KOMUNIKASI"
soup = BeautifulSoup(driver.page_source, 'html.parser')
target_link = None

rows = soup.select('table tbody tr')
for row in rows:
    data = {}
    cols = row.find_all('td')
    if len(cols) >= 3 and "INDUSTRI PENGOLAHAN" in cols[1].text:
        mikro_td = cols[2]  # Kolom ke-3 adalah 'mikro'
        button = mikro_td.find('button', class_='btn-link')
        if button and button.has_attr('href'):
            target_link = button['href']
            break

# BUKA LINK JIKA DITEMUKAN
if target_link:
    full_url = "https://satudata.umkm.go.id" + target_link if not target_link.startswith("http") else target_link
    print(f"🔗 Link mikro (INDUSTRI PENGOLAHAN): {full_url}")
    driver.get(full_url)
    time.sleep(5)

    # CARI TOMBOL VIEW
    view_links = driver.find_elements(By.PARTIAL_LINK_TEXT, "View")
    if view_links:
        href = view_links[0].get_attribute("href")
        print("🔍 View ditemukan:", href)

        # Simpan link mikro & view
        data['full_url_mikro'] = full_url
        data['view_link'] = href

        view_links[0].click()
        time.sleep(2)

        # Pindah ke tab baru jika ada
        if len(driver.window_handles) > 1:
            driver.switch_to.window(driver.window_handles[-1])
            print("✅ Pindah ke tab baru.")
        else:
            print("⚠️ Tab baru tidak terbuka.")

        # Tunggu sampai elemen latitude muncul
        try:
            wait.until(EC.presence_of_element_located((By.ID, "alamat_latitude")))
        except:
            print("⚠️ Elemen latitude tidak ditemukan dalam waktu tunggu. Coba tunggu manual 5 detik...")
            time.sleep(5)

        # Ambil data dari halaman view (tahap 1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        def get_input_value_by_id(id_):
            el = soup.find(id=id_)
            return el.get('value', '').strip() if el else ''

        from selenium.webdriver.support.ui import Select

        # Fungsi untuk mendapatkan bulan atau tahun yang terpilih berdasarkan atribut "selected"
        def get_selected_value(select_id):
            select_element = driver.find_element(By.ID, select_id)
            options = select_element.find_elements(By.TAG_NAME, "option")

            for option in options:
                if option.get_attribute("selected"):
                    return option.get_attribute("value"), option.text.strip()  # Ambil value dan nama bulan/tahun
            
            return None, None  # Jika tidak ada yang terpilih

        def get_textarea_value_by_id(id_):
            el = soup.find('textarea', {'id': id_})
            return el.text.strip() if el else ''

        def get_radio_value(name_):
            el = soup.find('input', {'name': name_, 'checked': True})
            if not el:
                return ''
            return 'Laki-laki' if el['value'] == 'L' else 'Perempuan'

        # Lokasi
        data['latitude'] = get_input_value_by_id('alamat_latitude')
        data['longitude'] = get_input_value_by_id('alamat_longitude')

        # Nama lengkap usaha

        nama_usaha_div = soup.find('label', {'for': 'nama_lengkap'})
        if nama_usaha_div:
            input_field = nama_usaha_div.find_next('input')
            data['nama_lengkap_usaha'] = input_field.get('value', '').strip() if input_field else ''
        else:
            data['nama_lengkap_usaha'] = ''

        # Alamat lengkap
        data['alamat_lengkap'] = get_textarea_value_by_id('alamat_lengkap')
        data['rt'] = get_input_value_by_id('alamat_rt')
        data['rw'] = get_input_value_by_id('alamat_rw')
        data['kecamatan'] = soup.find('label', {'for': 'alamat_id_kec'}).find_next('input').get('value', '').strip()
        data['kelurahan'] = soup.find('label', {'for': 'alamat_id_desa_kel'}).find_next('input').get('value', '').strip()

        # Klik tombol Tahap Selanjutnya pertama
        next_btn_1 = driver.find_element(By.ID, 'btn-next')
        next_btn_1.click()
        time.sleep(3)

        # Ambil data tahap 2
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        data['kegiatan_utama'] = get_textarea_value_by_id('kegiatan_utama')
        data['produk_utama'] = get_textarea_value_by_id('produk_utama')
        data['kategori_kbli'] = soup.find('label', {'for': 'id_jenis_kegiatan'}).find_next('input').get('value', '').strip()
        data['kode_kbli'] = soup.find('label', {'for': 'id_kbli'}).find_next('input').get('value', '').strip()
        data['status_badan_usaha'] = soup.find('label', {'for': 'id_status_badan_usaha'}).find_next('input').get('value', '').strip()

        # Mapping dari kode bulan ke nama bulan
        bulan_map = {
            "01": "Januari",
            "02": "Februari",
            "03": "Maret",
            "04": "April",
            "05": "Mei",
            "06": "Juni",
            "07": "Juli",
            "08": "Agustus",
            "09": "September",
            "10": "Oktober",
            "11": "November",
            "12": "Desember"
        }

        # Ambil bulan dan tahun yang terpilih
        bulan_value, _ = get_selected_value('bulan_mulai_operasi')
        tahun_value, tahun_text = get_selected_value('tahun_mulai_operasi')

        # Simpan nama bulan dari mapping jika value-nya valid
        if bulan_value and bulan_value in bulan_map:
            data['bulan_mulai_operasi'] = bulan_map[bulan_value]
        else:
            data['bulan_mulai_operasi'] = None

        if tahun_value and tahun_text:
            data['tahun_mulai_operasi'] = tahun_value
        else:
            data['tahun_mulai_operasi'] = None

        # Klik tombol Tahap Selanjutnya kedua
        next_btn_2 = driver.find_element(By.ID, 'btn-next')
        next_btn_2.click()
        time.sleep(3)

        # Ambil data pengusaha (tahap 3)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        data['nama_pengusaha'] = get_input_value_by_id('nama_pengusaha')
        data['jenis_kelamin'] = get_radio_value('id_jenis_kelamin')
        tanggal_lahir = get_input_value_by_id('tanggal_lahir')
        data['tahun_lahir'] = tanggal_lahir[:4] if tanggal_lahir else ''
        data['pendidikan'] = soup.find('label', {'for': 'id_pendidikan_formal'}).find_next('input').get('value', '').strip()
        data['anggota_koperasi'] = soup.find('label', {'for': 'is_anggota_koperasi'}).find_next('input').get('value', '').strip()

        # Klik tab Produksi & Pemasaran
        tab_produksi = driver.find_element(By.ID, 'step7-tab')
        tab_produksi.click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Produksi: Nilai Jual Produksi
        nilai_jual = soup.find('td', {'id': 'nilai_jual_total'})
        data['nilai_jual_produksi'] = nilai_jual.text.strip().replace('.', '') if nilai_jual else ''

        # Pendapatan operasional & non-operasional
        def get_input_value_by_name(name_):
            el = soup.find('input', {'name': name_})
            return el.get('value', '').strip() if el else ''

        data['pendapatan_operasional'] = get_input_value_by_name('pendapatan_ops')
        data['pendapatan_non_operasional'] = get_input_value_by_name('pendapatan_non_ops')

        # Wilayah pemasaran dalam negeri total
        wilayah_dn = soup.find('input', {'id': 'wilayah-dn'})
        data['wilayah_pemasaran_dalam_negeri_total'] = wilayah_dn.get('value', '').strip() if wilayah_dn else ''

        # Rincian wilayah pemasaran
        data['wilayah_pemasaran_desa'] = get_input_value_by_id('wilayah_1')
        data['wilayah_pemasaran_kecamatan'] = get_input_value_by_id('wilayah_2')
        data['wilayah_pemasaran_kabupaten'] = get_input_value_by_id('wilayah_3')

                # Tambahan Wilayah Pemasaran
        data['wilayah_pemasaran_kabkota_lain_provinsi'] = get_input_value_by_id('wilayah_4')
        data['wilayah_pemasaran_provinsi_lain'] = get_input_value_by_id('wilayah_5')
        data['wilayah_pemasaran_luar_negeri'] = get_input_value_by_id('wilayah-ln')

        # Omzet
        def get_input_value_by_name(name_):
            el = soup.find('input', {'name': name_})
            return el.get('value', '').strip() if el else ''

        data['omzet_dalam_negeri'] = get_input_value_by_name('omzet_dalam_negeri')
        data['omzet_luar_negeri'] = get_input_value_by_name('omzet_luar_negeri')

        # Metode Pemasaran
        def get_radio_status(ya_id, tidak_id):
            ya_input = soup.find('input', {'id': ya_id})
            tidak_input = soup.find('input', {'id': tidak_id})
            if ya_input and ya_input.has_attr('checked'):
                return 'ya'
            elif tidak_input and tidak_input.has_attr('checked'):
                return 'tidak'
            return ''

        data['metode_pemasaran_digital'] = get_radio_status('pemasaran-ya-1', 'pemasaran-tidak-1')
        data['metode_pemasaran_non_digital'] = get_radio_status('pemasaran-ya-2', 'pemasaran-tidak-2')
        data['metode_pemasaran_perantara'] = get_radio_status('pemasaran-ya-3', 'pemasaran-tidak-3')
        data['metode_pemasaran_vendor_pusat'] = get_radio_status('pemasaran-ya-4', 'pemasaran-tidak-4')
        data['metode_pemasaran_vendor_provinsi'] = get_radio_status('pemasaran-ya-5', 'pemasaran-tidak-5')
        data['metode_pemasaran_vendor_kabkota'] = get_radio_status('pemasaran-ya-6', 'pemasaran-tidak-6')

                # Klik tab Tenaga Kerja (VIII)
        tab_tenaga_kerja = driver.find_element(By.ID, 'step8-tab')
        tab_tenaga_kerja.click()
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # Total tenaga kerja dibayar
        total_tenaga_kerja_dibayar = 0
        tkdbayar_inputs = soup.select('.tkdbayar')
        for input_element in tkdbayar_inputs:
            value = input_element.get('value', '')
            if value.isdigit():
                total_tenaga_kerja_dibayar += int(value)
        data['tenaga_kerja_dibayar'] = total_tenaga_kerja_dibayar

        # Total tenaga kerja tidak dibayar
        total_tenaga_kerja_tidak_dibayar = 0
        tktbayar_inputs = soup.select('.tktbayar')
        for input_element in tktbayar_inputs:
            value = input_element.get('value', '')
            if value.isdigit():
                total_tenaga_kerja_tidak_dibayar += int(value)
        data['tenaga_kerja_tidak_dibayar'] = total_tenaga_kerja_tidak_dibayar

        # Klik tab XI: Laporan Keuangan (Pastikan ID-nya sesuai di situs, biasanya step11-tab)
        tab_laporan_keuangan = driver.find_element(By.ID, 'step11-tab')
        tab_laporan_keuangan.click()
        time.sleep(2)  # Tunggu tab terbuka

        # Ambil data laporan keuangan: "Apakah memiliki laporan keuangan usaha per 31 Desember?"
        try:
            input_el = driver.find_element(By.XPATH, "//label[@for='is_ada_lap_keu']/following-sibling::div//input")
            data['laporan_keuangan'] = input_el.get_attribute('value').strip()
        except:
            data['laporan_keuangan'] = ''

        # Simpan ke Excel
        df = pd.DataFrame([data])
        df.to_excel("data_umkm_lengkap.xlsx", index=False)

        print(f"\n✅ Data lengkap telah disimpan ke data_umkm_lengkap.xlsx")
    else:
        print("❌ Tidak menemukan tombol 'View'.")
else:
    print("❌ Tidak menemukan baris 'INFORMASI DAN KOMUNIKASI' atau tombol mikro.")

🔗 Link mikro (INDUSTRI PENGOLAHAN): https://satudata.umkm.go.id/admin/pl/umkm/detail?jenis_tabel=3&id_prov=15&id_kab=236&id_kec=3458&tipe_tabel=kegiatan&tipe_data=pl&id_kel=3&kolom=mikro
🔍 View ditemukan: https://satudata.umkm.go.id/admin/pl/umkm/pxbk6D2nVE7NPgR/review
✅ Pindah ke tab baru.
⚠️ Elemen latitude tidak ditemukan dalam waktu tunggu. Coba tunggu manual 5 detik...


AttributeError: 'NoneType' object has no attribute 'find_next'

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd

# Setup ChromeDriver
service = Service("D:/Tools/chromedriver-win64/chromedriver.exe")
driver = webdriver.Chrome(service=service)
wait = WebDriverWait(driver, 15)

# Login manual
driver.get("https://satudata.umkm.go.id/login")
input("➡️ Login dan isi captcha di browser, lalu tekan ENTER...")

# Akses halaman filter
driver.get("https://satudata.umkm.go.id/admin/pl/umkm")
time.sleep(3)

# PILIH TABEL
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#select2-jenis_tabel-container"))).click()
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field"))).send_keys("Tabel 1. Usaha Menurut Skala Usaha", Keys.ENTER)
time.sleep(1)

# PILIH MENURUT KEGIATAN USAHA
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='tipe_tabel_kegiatan']"))).click()
time.sleep(1)

# PILIH KATEGORI USAHA
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#select2-jenis_kegiatan-container"))).click()
search_input = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field")))

search_input.send_keys("G.")
search_input.send_keys(Keys.ENTER)
time.sleep(1)

# PILIH KABUPATEN
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#select2-alamat_id_kabkot-container"))).click()
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field"))).send_keys("KAB. PAMEKASAN", Keys.ENTER)
time.sleep(1)

# PILIH KECAMATAN
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#select2-alamat_id_kec-container"))).click()
wait.until(EC.presence_of_element_located((By.CLASS_NAME, "select2-search__field"))).send_keys("35.28.04 PAMEKASAN", Keys.ENTER)
time.sleep(1)

# CEKLIST PL KUMKM
wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "label[for='tipe_data_pl']"))).click()
time.sleep(1)

# KLIK FILTER
wait.until(EC.element_to_be_clickable((By.ID, "filter"))).click()
time.sleep(5)

# CARI LINK BUTTON PADA KOLOM MIKRO UNTUK BARIS "INFORMASI DAN KOMUNIKASI"
soup = BeautifulSoup(driver.page_source, 'html.parser')
target_link = None

rows = soup.select('table tbody tr')
for row in rows:
    data = {}
    cols = row.find_all('td')
    if len(cols) >= 3 and "PERDAGANGAN BESAR DAN ECERAN; REPARASI DAN PERAWATAN MOBIL DAN SEPEDA MOTOR" in cols[1].text:
        mikro_td = cols[2]  # Kolom ke-3 adalah 'mikro'
        button = mikro_td.find('button', class_='btn-link')
        if button and button.has_attr('href'):
            target_link = button['href']
            break

# BUKA LINK JIKA DITEMUKAN
if target_link:
    full_url = "https://satudata.umkm.go.id" + target_link if not target_link.startswith("http") else target_link
    print(f"🔗 Link mikro (PERDAGANGAN BESAR DAN ECERAN; REPARASI DAN PERAWATAN MOBIL DAN SEPEDA MOTOR): {full_url}")
    driver.get(full_url)
    time.sleep(5)

    # CARI TOMBOL VIEW
    view_links = driver.find_elements(By.PARTIAL_LINK_TEXT, "View")
    if view_links:
        # Ambil semua link 'View'
        view_urls = [view.get_attribute("href") for view in view_links]
        print(f"🔗 Total View links ditemukan: {len(view_urls)}")

        # Loop setiap link View
        for index, view_url in enumerate(view_urls):
            print(f"\n🔄 Scraping UMKM ke-{index+1} dari {len(view_urls)}")
            print(f"➡️  URL: {view_url}")

            try:
                driver.get(view_url)
                time.sleep(5)  # Tunggu halaman detailnya kebuka

                # ========== SCRAPING DIMULAI DI SINI ==========
                # Ambil data dari halaman view
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Fungsi untuk ambil nilai input berdasarkan ID
                def get_input_value_by_id(id_):
                    el = soup.find(id=id_)
                    return el.get('value', '').strip() if el else ''

                # Fungsi untuk ambil nilai textarea berdasarkan ID
                def get_textarea_value_by_id(id_):
                    el = soup.find('textarea', {'id': id_})
                    return el.text.strip() if el else ''

                # Fungsi untuk ambil nilai input berdasarkan name
                def get_input_value_by_name(name_):
                    el = soup.find('input', {'name': name_})
                    return el.get('value', '').strip() if el else ''

                # Fungsi untuk ambil radio button yang dicentang
                def get_radio_value(name_):
                    el = soup.find('input', {'name': name_, 'checked': True})
                    if not el:
                        return ''
                    return 'Laki-laki' if el['value'] == 'L' else 'Perempuan'
                
                # Fungsi untuk mendapatkan bulan atau tahun yang terpilih berdasarkan atribut "selected"
                def get_selected_value(select_id):
                    select_element = driver.find_element(By.ID, select_id)
                    options = select_element.find_elements(By.TAG_NAME, "option")

                    for option in options:
                        if option.get_attribute("selected"):
                            return option.get_attribute("value"), option.text.strip()  # Ambil value dan nama bulan/tahun
                    
                    return None, None  # Jika tidak ada yang terpilih

                # Ambil data lokasi
                data = {}
                data['latitude'] = get_input_value_by_id('alamat_latitude')
                data['longitude'] = get_input_value_by_id('alamat_longitude')

                # Nama lengkap usaha
                nama_usaha_div = soup.find('label', {'for': 'nama_lengkap'})
                if nama_usaha_div:
                    input_field = nama_usaha_div.find_next('input')
                    data['nama_lengkap_usaha'] = input_field.get('value', '').strip() if input_field else ''
                else:
                    data['nama_lengkap_usaha'] = ''

                # Alamat lengkap
                data['alamat_lengkap'] = get_textarea_value_by_id('alamat_lengkap')
                data['rt'] = get_input_value_by_id('alamat_rt')
                data['rw'] = get_input_value_by_id('alamat_rw')
                data['kecamatan'] = soup.find('label', {'for': 'alamat_id_kec'}).find_next('input').get('value', '').strip()
                data['kelurahan'] = soup.find('label', {'for': 'alamat_id_desa_kel'}).find_next('input').get('value', '').strip()

                # Klik tombol Tahap Selanjutnya pertama
                next_btn_1 = driver.find_element(By.ID, 'btn-next')
                next_btn_1.click()
                time.sleep(3)

                # Ambil data tahap 2
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                data['kegiatan_utama'] = get_textarea_value_by_id('kegiatan_utama')
                data['produk_utama'] = get_textarea_value_by_id('produk_utama')
                data['kategori_kbli'] = soup.find('label', {'for': 'id_jenis_kegiatan'}).find_next('input').get('value', '').strip()
                data['kode_kbli'] = soup.find('label', {'for': 'id_kbli'}).find_next('input').get('value', '').strip()
                data['status_badan_usaha'] = soup.find('label', {'for': 'id_status_badan_usaha'}).find_next('input').get('value', '').strip()

                # Mapping dari kode bulan ke nama bulan
                bulan_map = {
                    "01": "Januari",
                    "02": "Februari",
                    "03": "Maret",
                    "04": "April",
                    "05": "Mei",
                    "06": "Juni",
                    "07": "Juli",
                    "08": "Agustus",
                    "09": "September",
                    "10": "Oktober",
                    "11": "November",
                    "12": "Desember"
                }

                # Ambil bulan dan tahun yang terpilih
                bulan_value, _ = get_selected_value('bulan_mulai_operasi')
                tahun_value, tahun_text = get_selected_value('tahun_mulai_operasi')

                # Simpan nama bulan dari mapping jika value-nya valid
                if bulan_value and bulan_value in bulan_map:
                    data['bulan_mulai_operasi'] = bulan_map[bulan_value]
                else:
                    data['bulan_mulai_operasi'] = None

                if tahun_value and tahun_text:
                    data['tahun_mulai_operasi'] = tahun_value
                else:
                    data['tahun_mulai_operasi'] = None

                # Klik tombol Tahap Selanjutnya kedua
                next_btn_2 = driver.find_element(By.ID, 'btn-next')
                next_btn_2.click()
                time.sleep(3)

                # Ambil data pengusaha (tahap 3)
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                data['nama_pengusaha'] = get_input_value_by_id('nama_pengusaha')
                data['jenis_kelamin'] = get_radio_value('id_jenis_kelamin')
                tanggal_lahir = get_input_value_by_id('tanggal_lahir')
                data['tahun_lahir'] = tanggal_lahir[:4] if tanggal_lahir else ''
                data['pendidikan'] = soup.find('label', {'for': 'id_pendidikan_formal'}).find_next('input').get('value', '').strip()
                data['anggota_koperasi'] = soup.find('label', {'for': 'is_anggota_koperasi'}).find_next('input').get('value', '').strip()

                # Klik tab Produksi & Pemasaran
                tab_produksi = driver.find_element(By.ID, 'step7-tab')
                tab_produksi.click()
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Produksi: Nilai Jual Produksi
                nilai_jual = soup.find('td', {'id': 'nilai_jual_total'})
                data['nilai_jual_produksi'] = nilai_jual.text.strip().replace('.', '') if nilai_jual else ''

                # Pendapatan operasional & non-operasional
                def get_input_value_by_name(name_):
                    el = soup.find('input', {'name': name_})
                    return el.get('value', '').strip() if el else ''

                data['pendapatan_operasional'] = get_input_value_by_name('pendapatan_ops')
                data['pendapatan_non_operasional'] = get_input_value_by_name('pendapatan_non_ops')

                # Wilayah pemasaran dalam negeri total
                wilayah_dn = soup.find('input', {'id': 'wilayah-dn'})
                data['wilayah_pemasaran_dalam_negeri_total'] = wilayah_dn.get('value', '').strip() if wilayah_dn else ''

                # Rincian wilayah pemasaran
                data['wilayah_pemasaran_desa'] = get_input_value_by_id('wilayah_1')
                data['wilayah_pemasaran_kecamatan'] = get_input_value_by_id('wilayah_2')
                data['wilayah_pemasaran_kabupaten'] = get_input_value_by_id('wilayah_3')

                        # Tambahan Wilayah Pemasaran
                data['wilayah_pemasaran_kabkota_lain_provinsi'] = get_input_value_by_id('wilayah_4')
                data['wilayah_pemasaran_provinsi_lain'] = get_input_value_by_id('wilayah_5')
                data['wilayah_pemasaran_luar_negeri'] = get_input_value_by_id('wilayah-ln')

                # Omzet
                def get_input_value_by_name(name_):
                    el = soup.find('input', {'name': name_})
                    return el.get('value', '').strip() if el else ''

                data['omzet_dalam_negeri'] = get_input_value_by_name('omzet_dalam_negeri')
                data['omzet_luar_negeri'] = get_input_value_by_name('omzet_luar_negeri')

                # Metode Pemasaran
                def get_radio_status(ya_id, tidak_id):
                    ya_input = soup.find('input', {'id': ya_id})
                    tidak_input = soup.find('input', {'id': tidak_id})
                    if ya_input and ya_input.has_attr('checked'):
                        return 'ya'
                    elif tidak_input and tidak_input.has_attr('checked'):
                        return 'tidak'
                    return ''

                data['metode_pemasaran_digital'] = get_radio_status('pemasaran-ya-1', 'pemasaran-tidak-1')
                data['metode_pemasaran_non_digital'] = get_radio_status('pemasaran-ya-2', 'pemasaran-tidak-2')
                data['metode_pemasaran_perantara'] = get_radio_status('pemasaran-ya-3', 'pemasaran-tidak-3')
                data['metode_pemasaran_vendor_pusat'] = get_radio_status('pemasaran-ya-4', 'pemasaran-tidak-4')
                data['metode_pemasaran_vendor_provinsi'] = get_radio_status('pemasaran-ya-5', 'pemasaran-tidak-5')
                data['metode_pemasaran_vendor_kabkota'] = get_radio_status('pemasaran-ya-6', 'pemasaran-tidak-6')

                        # Klik tab Tenaga Kerja (VIII)
                tab_tenaga_kerja = driver.find_element(By.ID, 'step8-tab')
                tab_tenaga_kerja.click()
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source, 'html.parser')

                # Total tenaga kerja dibayar
                total_tenaga_kerja_dibayar = 0
                tkdbayar_inputs = soup.select('.tkdbayar')
                for input_element in tkdbayar_inputs:
                    value = input_element.get('value', '')
                    if value.isdigit():
                        total_tenaga_kerja_dibayar += int(value)
                data['tenaga_kerja_dibayar'] = total_tenaga_kerja_dibayar

                # Total tenaga kerja tidak dibayar
                total_tenaga_kerja_tidak_dibayar = 0
                tktbayar_inputs = soup.select('.tktbayar')
                for input_element in tktbayar_inputs:
                    value = input_element.get('value', '')
                    if value.isdigit():
                        total_tenaga_kerja_tidak_dibayar += int(value)
                data['tenaga_kerja_tidak_dibayar'] = total_tenaga_kerja_tidak_dibayar

                # Klik tab XI: Laporan Keuangan (Pastikan ID-nya sesuai di situs, biasanya step11-tab)
                tab_laporan_keuangan = driver.find_element(By.ID, 'step11-tab')
                tab_laporan_keuangan.click()
                time.sleep(2)  # Tunggu tab terbuka

                # Ambil data laporan keuangan: "Apakah memiliki laporan keuangan usaha per 31 Desember?"
                try:
                    input_el = driver.find_element(By.XPATH, "//label[@for='is_ada_lap_keu']/following-sibling::div//input")
                    data['laporan_keuangan'] = input_el.get_attribute('value').strip()
                except:
                    data['laporan_keuangan'] = ''

                import os
                # Simpan ke Excel secara bertahap (append jika file sudah ada)
                df = pd.DataFrame([data])
                file_path = "data_umkm_lengkap.xlsx"

                if os.path.exists(file_path):
                    df_existing = pd.read_excel(file_path)
                    df_combined = pd.concat([df_existing, df], ignore_index=True)
                    df_combined.to_excel(file_path, index=False)
                else:
                    df.to_excel(file_path, index=False)


                print(f"✅ Data UMKM ke-{index+1} telah disimpan ke data_umkm_lengkap.xlsx")

            except Exception as e:
                print(f"❗ Gagal membuka/scraping link ke-{index+1}: {e}")
    else:
        print("❌ Tidak menemukan tombol 'View'.")
else:
    print("❌ Tidak menemukan baris 'INFORMASI DAN KOMUNIKASI' atau tombol mikro.")


🔗 Link mikro (PERDAGANGAN BESAR DAN ECERAN; REPARASI DAN PERAWATAN MOBIL DAN SEPEDA MOTOR): https://satudata.umkm.go.id/admin/pl/umkm/detail?jenis_tabel=3&id_prov=15&id_kab=255&id_kec=3842&tipe_tabel=kegiatan&tipe_data=pl&id_kel=7&kolom=mikro
🔗 Total View links ditemukan: 9957

🔄 Scraping UMKM ke-1 dari 9957
➡️  URL: https://satudata.umkm.go.id/admin/pl/umkm/j74Ylr3JYVNnK6D/review
✅ Data UMKM ke-1 telah disimpan ke data_umkm_lengkap.xlsx

🔄 Scraping UMKM ke-2 dari 9957
➡️  URL: https://satudata.umkm.go.id/admin/pl/umkm/Oy6vXprBgD5lZa9/review
✅ Data UMKM ke-2 telah disimpan ke data_umkm_lengkap.xlsx

🔄 Scraping UMKM ke-3 dari 9957
➡️  URL: https://satudata.umkm.go.id/admin/pl/umkm/72aRk3DoQEyl6ZO/review
✅ Data UMKM ke-3 telah disimpan ke data_umkm_lengkap.xlsx

🔄 Scraping UMKM ke-4 dari 9957
➡️  URL: https://satudata.umkm.go.id/admin/pl/umkm/75E9kJp3BYNABRP/review
✅ Data UMKM ke-4 telah disimpan ke data_umkm_lengkap.xlsx

🔄 Scraping UMKM ke-5 dari 9957
➡️  URL: https://satudata.umkm.go.

KeyboardInterrupt: 